In [342]:
#Importing needed libraries
import requests
import pandas as pd
from datetime import datetime
import geopandas as gpd
import fiona
import time
from json import JSONDecodeError
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.cm as cm
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
from statsmodels.discrete.count_model import ZeroInflatedPoisson
from personal_lib import general_functions as gf
import re
from typing import Optional, List, Tuple
import googlemaps
import os
import math
import pyogrio
import geopandas as gpd
import io



creds = gf.get_creds()
google_api_key = creds["Google"]["geocoding"]
open_ai = creds["openai"]["FirstTestKey"]
gmaps = googlemaps.Client(key=google_api_key)

STATE_ABBR = "NY"
pd.options.display.max_columns = 500 
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="capstone_test")

### NYC ENERGY DATA BUILDINGS (### Pulling in the Data Via the APi online Local Law 84) 

##### Skip here until reading in files from CSV, because this takes very long to run.

In [211]:
## There are mulitple sources of yearly data. Compiling the source URLs here as well as the API urls to grab
## privately owned buildings over 25,000 ft2 and in City-owned buildings over 10,000 ft2
building_energy_LL84_sources={"2022+":{
    "api":"https://data.cityofnewyork.us/resource/5zyy-y8am.json",
    "info":"https://data.cityofnewyork.us/Environment/NYC-Building-Energy-and-Water-Data-Disclosure-for-/5zyy-y8am/about_data"
                              },
                              "2021":{
    "api":"https://data.cityofnewyork.us/resource/7x5e-2fxh.json",
    "info":"https://data.cityofnewyork.us/Environment/Energy-and-Water-Data-Disclosure-for-Local-Law-84-/7x5e-2fxh/about_data"
                              },
                              "2020":{
    "api":"https://data.cityofnewyork.us/resource/usc3-8zwd.json",
    "info":"https://data.cityofnewyork.us/Environment/Energy-and-Water-Data-Disclosure-for-Local-Law-84-/usc3-8zwd/about_data"
                                  },
                              "2019":{
    "api":"https://data.cityofnewyork.us/resource/wcm8-aq5w.json",
    "info":"https://data.cityofnewyork.us/Environment/Energy-and-Water-Data-Disclosure-for-Local-Law-84-/wcm8-aq5w/about_data"
                                  },
                              "2018":{
    "api":"https://data.cityofnewyork.us/resource/4tys-3tzj.json",
    "info":"https://data.cityofnewyork.us/Environment/Energy-and-Water-Data-Disclosure-for-Local-Law-84-/4tys-3tzj/about_data"
                                   },
                              "2017":{
    "api":"https://data.cityofnewyork.us/resource/4t62-jm4m.json",
    "info":"https://data.cityofnewyork.us/Environment/Energy-and-Water-Data-Disclosure-for-Local-Law-84-/4t62-jm4m/about_data"
                                  },
                              "2016":{
    "api":"https://data.cityofnewyork.us/resource/utpj-74fz.json",
    "info":"https://data.cityofnewyork.us/Environment/Energy-and-Water-Data-Disclosure-for-Local-Law-84-/utpj-74fz/about_data"
                                  },
                              "2015":{
    "api":"https://data.cityofnewyork.us/resource/77q4-nkfh.json",
    "info":"https://data.cityofnewyork.us/Environment/Energy-and-Water-Data-Disclosure-for-Local-Law-84-/77q4-nkfh/about_data"
                                  },
                              "2014":{
    "api":"https://data.cityofnewyork.us/resource/nbun-wekj.json",
    "info":"https://data.cityofnewyork.us/Environment/Energy-and-Water-Data-Disclosure-for-Local-Law-84-/nbun-wekj/about_data"
                                  },
                              "2013":{
    "api":"https://data.cityofnewyork.us/resource/yr5p-wjer.json",
    "info":"http://data.cityofnewyork.us/Environment/Energy-and-Water-Data-Disclosure-for-Local-Law-84-/yr5p-wjer"
                                  },
                              "2012":{
    "api":"https://data.cityofnewyork.us/resource/r6ub-zhff.json",
    "info":"https://data.cityofnewyork.us/Environment/Energy-and-Water-Data-Disclosure-for-Local-Law-84-/r6ub-zhff/about_data"
                                  },
                              "2011":{
    "api":"https://data.cityofnewyork.us/resource/k7nh-aufb.json",
    "info":"https://data.cityofnewyork.us/Environment/Energy-and-Water-Data-Disclosure-for-Local-Law-84-/k7nh-aufb/about_data"
                                  },
                              "2010":{
    "api":"https://data.cityofnewyork.us/resource/kswi-37bp.json",
    "info":"https://data.cityofnewyork.us/Environment/Energy-and-Water-Data-Disclosure-for-Local-Law-84-/kswi-37bp/about_data"
                                  }
}

#Monthly Data? 2018-2023
    #                           "Monthly":{
    # "api":"https://data.cityofnewyork.us/resource/fvp3-gcb2.json",
    # "info":"https://data.cityofnewyork.us/Environment/Local-Law-84-Monthly-Data/fvp3-gcb2/about_data"
    # }}


In [ ]:
PAGE = 1000               # <-- enforce 1,000 rows per page
TIMEOUT = 30
MAX_RETRIES = 5
BACKOFF_BASE = 1.5

session = requests.Session()
headers = {}

def fetch_all_rows_1k(api_url: str, source_years: str, source_info_url: str) -> pd.DataFrame:
    offset = 0
    frames = []

    while True:
        params = {"$limit": PAGE, "$offset": offset}

        for attempt in range(1, MAX_RETRIES + 1):
            try:
                resp = session.get(api_url, params=params, headers=headers, timeout=TIMEOUT)
                if resp.status_code in (429, 502, 503, 504):
                    time.sleep(BACKOFF_BASE ** attempt * (0.1 * attempt))
                    continue
                resp.raise_for_status()

                if "json" not in resp.headers.get("Content-Type", "").lower():
                    preview = resp.text[:200]
                    raise ValueError(f"Non-JSON response (status {resp.status_code}): {preview}")

                data_chunk = resp.json()
                if not data_chunk:
                    return pd.concat(frames, ignore_index=True) if frames else pd.DataFrame()

                # Create DataFrame
                df = pd.DataFrame(data_chunk)

                # 🔑 Add your metadata columns here
                df["source_years"] = source_years
                df["source_api_url"] = api_url
                df["source_info_url"] = source_info_url

                frames.append(df)

                # If less than PAGE, stop; otherwise keep paginating
                if len(data_chunk) < PAGE:
                    return pd.concat(frames, ignore_index=True)

                offset += PAGE
                break  # Success, go to next page

            except (requests.RequestException, JSONDecodeError, ValueError) as e:
                if attempt == MAX_RETRIES:
                    print(f"⚠️ Failed fetching {api_url} at offset {offset}: {e}")
                    return pd.concat(frames, ignore_index=True) if frames else pd.DataFrame()
                time.sleep(BACKOFF_BASE ** attempt * (0.1 * attempt))


In [ ]:
agg_running_list = []
for k, v in building_energy_LL84_sources.items():
    print(f"Fetching {k} -> {v['api']}")
    df = fetch_all_rows_1k(v["api"], k, v["info"])
    if not df.empty:
        agg_running_list.append(df)
    else:
        print(f"Warning: no rows returned for {k} ({v['api']}).")

nyc_building_energy = pd.concat(agg_running_list, ignore_index=True) if agg_running_list else pd.DataFrame()
print(f"Total rows: {len(nyc_building_energy)}")

In [ ]:
## Formatting into proper df
# print(nyc_building_energy.shape)
## Saving to File
# nyc_building_energy.to_csv("nyc_dob_energy_2010_2024.csv",index=False)

#### End Skip here. Pick up and read in data

In [2]:
### If just want residential limist continue to skip in untill reading in residential 3
# nyc_building_energy=  pd.read_csv("nyc_dob_energy_2010_2024.csv")

C:\Users\johnf\AppData\Local\Temp\ipykernel_62516\3954150524.py:2: DtypeWarning: Columns (2,3,4,5,6,7,10,11,12,13,14,15,16,17,18,19,20,21,22,23,25,26,27,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,244,245,252,253,254,257,258,259,260,261,262,263,264,268,269,270,271,272,27

In [38]:
### Columns to Drop 
to_drop =[
## Banking oriented COlumns
'bank_branch_computer_density',
 'bank_branch_gross_floor_area',
 'bank_branch_gross_floor_area_ft',
 'bank_branch_number_of',
 'bank_branch_number_of_workers',
 'bank_branch_percent_that',
 'bank_branch_weekly_operating',
 'bank_branch_worker_density',
## College / Uni / School
'college_university_gross',
 'college_university_gross_floor_area_ft',
 'college_university_number',
    'k_12_school_computer_density',
 'k_12_school_cooking_facilities',
 'k_12_school_gross_floor_area',
 'k_12_school_gross_floor_area_ft',
 'k_12_school_high_school',
 'k_12_school_percent_that',
 'k_12_school_refrigeration',
 'k_12_school_weekend_operation',
 'laboratory_gross_floor_area_ft',
    'library_gross_floor_area',
# OTher
"automobile_dealership_gross",
'convenience_store_without',
'data_center_energy_estimates_applied',
 'data_center_gross_floor_area',
 'data_center_gross_floor_area_ft',
 'data_center_it_energy',
 'data_center_it_energy_configuration',
 'data_center_it_equipment_input_meter_kwh',
 'data_center_it_site_energy',
 'data_center_it_site_energy_kwh',
 'data_center_it_source_energy_kbtu',
 'data_center_national_median',
 'data_center_pdu_input_meter_kwh',
 'data_center_pdu_output_meter_kwh',
 'data_center_ups_output_meter_kwh',
 'enclosed_mall_gross_floor',
 'enclosed_mall_gross_floor_area_ft',
 'fast_food_restaurant_gross',
 'financial_office_gross_floor',
 'financial_office_gross_floor_area_ft',
 'financial_office_number_of',
 'financial_office_number_of_1',
 'financial_office_number_of_computers',
 'financial_office_number_of_workers_on_main_shift',
 'financial_office_weekly',
 'financial_office_weekly_operating_hours',
 'fitness_center_health_club',
 'fitness_center_health_club_gym_gross_floor_area_ft',
 'food_sales_gross_floor_area',
 'food_sales_gross_floor_area_ft',
 'food_service_gross_floor',
 'food_service_gross_floor_area_ft',
     'worship_facility_computer',
 'worship_facility_cooking',
 'worship_facility_gross_floor',
 'worship_facility_gross_floor_area_ft',
 'worship_facility_weekly',
    'supermarket_grocery_cooking',
 'supermarket_grocery_gross',
 'supermarket_grocery_gross_floor_area_ft',
 'supermarket_grocery_number',
 'supermarket_grocery_number_1',
 'supermarket_grocery_number_2',
 'supermarket_grocery_number_of_open_or_closed_refrigeration_freezer_units',
 'supermarket_grocery_number_of_walk_in_refrigeration_freezer_units',
 'supermarket_grocery_percent',
 'supermarket_grocery_walk',
 'supermarket_grocery_weekly',
 'supermarket_grocery_worker',
 'swimming_pool_approximate',
 'swimming_pool_location_of',
 'swimming_pool_months_in_use',
     'social_meeting_hall_gross',
 'social_meeting_hall_gross_floor_area_ft',
     'residence_hall_dormitory',
 'residence_hall_dormitory_1',
 'residence_hall_dormitory_2',
 'residence_hall_dormitory_3',
 'residence_hall_dormitory_4',
 'residence_hall_dormitory_gross_floor_area_ft',
 'restaurant_gross_floor_area',
 'restaurant_gross_floor_area_ft',
 'restaurant_weekly_operating',
 'restaurant_weekly_operating_hours',
 'restaurant_worker_density',
 'restaurant_worker_density_number_per_1_000_sq_ft',
 'retail_store_cash_register',
 'retail_store_computer_density',
 'retail_store_exterior_entrance',
 'retail_store_gross_floor',
 'retail_store_gross_floor_area_ft',
 'retail_store_number_of_open',
 'retail_store_number_of_open_or_closed_refrigeration_freezer_units',
 'retail_store_number_of_walk',
 'retail_store_number_of_walk_in_refrigeration_freezer_units',
 'retail_store_open_or_closed',
 'retail_store_percent_that',
 'retail_store_walk_in',
 'retail_store_weekly_operating',
 'retail_store_worker_density',
 'self_storage_facility_gross',
 'self_storage_facility_gross_floor_area_ft',
 'senior_care_community_average',
 'senior_care_community_gross',
 'senior_care_community_maximum',
 'senior_care_community_number',
 'senior_care_community_number_1',
 'senior_care_community_number_2',
 'senior_care_community_number_3',
 'senior_care_community_number_4',
 'senior_care_community_number_5',
 'senior_care_community_number_6',
 'senior_care_community_percent',
 'senior_living_community_gross_floor_area_ft',
 'senior_living_community_living_unit_density_number_per_1_000_sq_ft',
    'movie_theater_gross_floor_area_ft',
    'non_refrigerated_warehouse',
 'non_refrigerated_warehouse_1',
 'non_refrigerated_warehouse_2',
 'non_refrigerated_warehouse_3',
 'non_refrigerated_warehouse_4',
 'non_refrigerated_warehouse_5',
 'non_refrigerated_warehouse_gross_floor_area_ft',
     'parking_completely_enclosed',
 'parking_completely_enclosed_parking_garage_size_ft',
 'parking_gross_floor_area',
 'parking_gross_floor_area_ft',
 'parking_open_parking_lot',
 'parking_open_parking_lot_size_ft',
 'parking_partially_enclosed',
 'parking_partially_enclosed_parking_garage_size_ft',
     'strip_mall_gross_floor_area',
## Hostpial / Hotel
    'hospital_general_medical',
 'hospital_general_medical_1',
 'hospital_general_medical_10',
 'hospital_general_medical_11',
 'hospital_general_medical_12',
 'hospital_general_medical_13',
 'hospital_general_medical_14',
 'hospital_general_medical_15',
 'hospital_general_medical_16',
 'hospital_general_medical_17',
 'hospital_general_medical_2',
 'hospital_general_medical_3',
 'hospital_general_medical_4',
 'hospital_general_medical_5',
 'hospital_general_medical_6',
 'hospital_general_medical_7',
 'hospital_general_medical_8',
 'hospital_general_medical_9',
 'hotel_amount_of_laundry',
 'hotel_cooking_facilities',
 'hotel_full_service_spa_floor',
 'hotel_gross_floor_area_ft',
 'hotel_gym_fitness_center',
 'hotel_gym_fitness_center_floor_area_ft',
 'hotel_number_of_rooms',
 'hotel_percent_that_can_be',
 'hotel_room_density_number',
 'hotel_type_of_laundry_facility',
 'hotel_worker_density_number',
 'urgent_care_clinic_other',
 'urgent_care_clinic_other_outpatient_gross_floor_area_ft',
    'mailing_center_post_office_gross_floor_area_ft',
 'manufacturing_industrial_plant_gross_floor_area_ft',
 'medical_office_gross_floor',
 'medical_office_gross_floor_area_ft',
 'medical_office_mri_machine',
 'medical_office_number_of',
 'medical_office_number_of_1',
 'medical_office_number_of_computers',
 'medical_office_number_of_mri_machines',
 'medical_office_number_of_workers_on_main_shift',
 'medical_office_percent_that',
 'medical_office_percent_that_1',
 'medical_office_percent_that_can_be_cooled',
 'medical_office_percent_that_can_be_heated',
 'medical_office_weekly',
 'medical_office_weekly_operating_hours',
    'museum_gross_floor_area_ft',
    'office_computer_density_number',
 'office_gross_floor_area_ft',
 'office_number_of_computers',
 'office_number_of_workers',
 'office_number_of_workers_on_main_shift',
 'office_percent_that_can_be',
 'office_percent_that_can_be_1',
 'office_percent_that_can_be_cooled',
 'office_percent_that_can_be_heated',
 'office_weekly_operating_hours',
 'office_worker_density_number',
 'office_worker_density_number_per_1_000_sq_ft',
     'adult_education_gross_floor',
 'adult_education_gross_floor_area_ft',
    'data_center_ups_output_meter',
 'data_center_pdu_input_meter',
 'data_center_pdu_output_meter',
 'data_center_it_equipment',
 'data_center_it_site_energy',
 'data_center_it_source_energy',
 'data_center_pue',
 'data_center_national_median',
 'data_center_gross_floor_area',
 'data_center_ups_system',
 'data_center_it_energy',
 'data_center_cooling_equipment',
'supermarkets_grocery_gross',
 'supermarkets_grocery_number',
 'supermarkets_grocery_number_1',
 'supermarkets_grocery_number_2',
 'supermarkets_grocery_percent',
 'supermarkets_grocery_presence',
 'supermarkets_grocery_walk',
 'supermarkets_grocery_weekly',
 'supermarkets_grocery_workers',
'house_of_worship_gross_floor',
 'house_of_worship_pc_density',
 'house_of_worship_weekly',
 'house_of_worship_presence',
'residence_halls_dormitories',
'medical_office_percent_cooled',
 'residence_halls_dormitories_1',
 'residence_halls_dormitories_2',
 'residence_halls_dormitories_3',
 'residence_halls_dormitories_4',
'hotel_onsite_laundry_short',
 'warehouse_unrefrigerated',
 'warehouse_unrefrigerated_1',
 'warehouse_unrefrigerated_2',
 'warehouse_unrefrigerated_3',
 'warehouse_unrefrigerated_4',
 'warehouse_unrefrigerated_5',
 'warehouse_unrefrigerated_6',
 'hospital_gross_floor_area',
 'hospital_laboratory_y_1_n',
 'hospital_laundry_facility',
 'hospital_maximum_number_of',
 'hospital_number_of_buildings',
 'warehouse_refrigerated_gross',
 'warehouse_refrigerated_weekly',
 'warehouse_refrigerated_workers',
 'hospital_number_of_licensed',
 'multifamily_home_dishwashers',
 'fuel_oil_2_use_kbtu',
 'fuel_oil_4_use_kbtu',
 'diesel_2_use_kbtu',
 'district_steam_use_kbtu',
 'natural_gas_use_kbtu',
'natural_gas_use_therms',
 'difference_from_national',
 'difference_from_national_1',
 'current_direct_ghg_emissions',
 'current_indirect_ghg_emissions',
'office_gross_floor_area_sq',
 'office_office_air_conditioned',
 'office_pc_density',
 'office_workers_density',
 'parking_enclosed_floor_area',
 'parking_non_enclosed_floor',
 'parking_open_floor_area_w',
 'parking_weekly_hours_of_access',
 'retail_gross_floor_area_sq',
 'retail_cash_register_density',
 'retail_exterior_entrance',
 'retail_workers_density',
 'retail_weekly_operating_hours',
 'retail_walk_in_refrig_density',
 'retail_percent_cooled',
 'retail_pc_density',
 'retail_open_closed_refrig',
 'retail_number_of_walk_in',
 'retail_number_of_open_or',
'data_center_annual_it_energy',
'other_space_type_name',
 'other_gross_floor_area_sq',
 'other_number_of_pcs',
 'other_workers_on_main_shift',
 'hotel_gross_floor_area_sq',
 'hotel_room_density',
 'hotel_percent_cooled',
 'hotel_presence_of_cooking',
 'hotel_workers_density',
 'bank_financial_institution',
 'bank_financial_institution_1',
 'bank_financial_institution_2',
 'bank_financial_institution_3',
 'bank_financial_institution_4',
 'bank_financial_institution_5',
 'k_12_school_high_school_y',
 'k_12_school_open_weekends',
 'k_12_school_pc_density',
 'k_12_school_percent_cooled',
 'k_12_school_walk_in_refrig',
 'k_12_school_presence_of',
 'swimming_pool_size',
 'swimming_pool_indoor_outdoor',
 'hotel_floor_area_of_full',
 'hotel_floor_area_of_gym',
 'hotel_average_occupancy',
 'hotel_quantity_of_laundry',
'number_of_staffed_beds',
 'number_of_mri_machines',
 'senior_care_facility_average',
 'senior_care_facility_gross',
 'senior_care_facility_number',
 'senior_care_facility_number_1',
 'senior_care_facility_number_2',
 'senior_care_facility_number_3',
 'senior_care_facility_number_4',
 'senior_care_facility_percent',
 'senior_care_facility_total',
 'senior_care_facility_total_1',
 'senior_care_facility_workers',]

In [185]:
chunksize = 100000  
filtered_chunks = []
for chunk in pd.read_csv("nyc_dob_energy_2010_2024.csv", chunksize=chunksize, usecols=lambda col: col not in to_drop):
    ## Using nulls instead of "not Available".
        ## Using nulls instead of "not Available".
    chunk = chunk.replace("Not Available",np.nan)
    # Convert year_ending to datetime
    chunk["year_ending"] = pd.to_datetime(chunk["year_ending"], errors="coerce")
    # Optional: extract just the year if that’s all you need
    chunk["year_ending_year"] = chunk["year_ending"].dt.year
    # print(chunk["year_ending_year"].unique())
    
    ### Property Types
    chunk["primary_property_type"] = chunk["primary_property_type"].combine_first(chunk["primary_property_type_epa"])
    chunk["primary_property_type_self"] = chunk["primary_property_type_self"].combine_first(chunk["primary_property_type_self_selected"])
    chunk = chunk.drop(columns=["primary_property_type_self_selected","primary_property_type_epa"])


    ## Beginning of Limiting to Multifamily Homes (Step 1)
    residential_chunk = chunk[(((chunk["list_of_all_property_use"].isin([i for i in chunk["list_of_all_property_use"].unique() if 'Multifamily Housing' in str(i)]))
            |(chunk["list_of_all_property_use"].isnull()))
           &((chunk["primary_property_type_self"]=="Multifamily Housing")|(chunk["primary_property_type_self"].isnull()))
           &((chunk["primary_property_type"]=="Multifamily Housing")|(chunk["primary_property_type"].isnull())))]
    
    ## Second Step is limiting to NON MIXED USE, so ONLY MultiFamily Residences
    residential_chunk2 = residential_chunk[((residential_chunk['list_of_all_property_use'].isnull())
                                            | residential_chunk['list_of_all_property_use'].apply(
                                                lambda x: len(x.split(",")) == 1 if isinstance(x, str) else False
                                            ))]


    ## Limtiing to the Metered Areas for whole property or whole building
    residential_chunk3 = residential_chunk2[((residential_chunk2["metered_areas_energy"].isin(['Whole Building', "Whole Property"]))
                                             |(residential_chunk2["metered_areas_energy"].isnull()))]

    residential_chunk3 = residential_chunk3[((residential_chunk3["construction_status"]=='Existing')|(residential_chunk3["construction_status"].isnull()))]
    
    ## Stand Alone Properties
    residential_chunk3 = residential_chunk3[((residential_chunk3["parent_property_id"].isnull())
                                             |(residential_chunk3["parent_property_id"]=='Not Applicable: Standalone Property'))]
    ## Dropping what i already filtered on
    residential_chunk3 = residential_chunk3.drop(columns=["primary_property_type_self","primary_property_type",
                                                          "national_median_reference", "list_of_all_property_use",
                                                          "largest_property_use_type","construction_status",
                                                          "parent_property_id","parent_property_name"])
    
    # ## Fruther Limiting to those that are under 10 Stories. 
    # residential_chunk2 = residential_chunk2.replace("Not Available",np.nan).dropna(how='all',axis=1)
    # residential_chunk2 = residential_chunk2.dropna(how='all',axis=1)

    filtered_chunks.append(residential_chunk3)

residential3 = pd.concat(filtered_chunks, ignore_index=True)

C:\Users\johnf\AppData\Local\Temp\ipykernel_62516\1837888054.py:3: DtypeWarning: Columns (3,6,7,10,11,12,13,14,16,17,18,19,20,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,53,56,58,59,60,61,62,63,64,65,67,68,71,72,73,74,75,76,77,78,79,80,81,82,83,85,87,88,91,93,94,95,97,98,99,100,101,102,103,104,105,106,107,108,109,121,124,127,129,130,131,133,155,159,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,185,196,208,209,214,215,216,220,221,222,223,224,225,226,227,228,229,230,232,233,234,235,236,237,238,239,244,245,252,253,254,257,258,259,260,261,262,263,264,268,269,270,272,273,274,275,276,277,278,279,280,281,283,284,285,287,288,289,290,291,292,293,294,295,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,337,338,340,342,343,345,346,355,356,358,366,367,368,369,370,371,372,373,374,375,376,377,385,389,394,395,396,397,398,399,400,401,402,403,404,405,406,407,4

In [5]:
# residential3 = pd.read_csv("residential3.csv") 
# residential3.to_csv("residential3.csv", index=False) 

### Fixing Via Self Joins and Otherwise for Earlier years in data without Property ID

In [186]:
## Finalize the truth set year column 
residential3["year_ending_year"] = residential3["year_ending_year"].combine_first(residential3["source_years"]).astype(int)
### Consolidating those columns that contain potential ID numbers
residential3["bbl"] = residential3["nyc_borough_block_and_lot"].combine_first(residential3['nyc_borough_block_and_lot_bbl']).combine_first(residential3["bbl10"]).combine_first(residential3["bbl"])
residential3.loc[residential3["bbl"].notna(), "bbl"] = residential3.loc[residential3["bbl"].notna(), "bbl"].astype(str).str.replace(r"\.0$", "", regex=True).str.replace(r"-", "", regex=False).str.replace(r"/", "", regex=False).str.replace(r".", "", regex=False)
residential3["postcode"] = residential3["postcode"].combine_first(residential3["postal_code"])
residential3['postcode'] = residential3['postcode'].apply(lambda x : str(x).replace(",0",'') if x else x)
residential3 = residential3.drop(columns=["postal_code","nyc_borough_block_and_lot",'nyc_borough_block_and_lot_bbl',"bbl10"])

In [187]:
### Checking the row numbers and Property Id vals
residential3.groupby(["year_ending_year"]).agg({"property_id":["nunique","count"],"source_api_url":"count"})

property_id        source_api_url
                     nunique  count          count
year_ending_year                                  
2010                       0      0          10329
2011                       0      0          12400
2012                     804    804            804
2013                    9291   9405           9405
2014                    9416   9435           9435
2015                    6834   6908           6908
2016                    7059   7071           7071
2017                   11948  18323          18323
2018                   13378  13696          13696
2019                   13689  14367          14367
2020                   17205  17290          17290
2021                   13054  13167          13167
2022                   13401  13444          13444
2023                   14574  14803          14803

In [190]:
### Attempting to self-enrich data for property ids
null_property_ids = residential3[residential3["property_id"].isnull()]
print(null_property_ids.shape)
property_ids = residential3[~residential3["property_id"].isnull()]
print(property_ids.shape)

(22729, 520)
(138713, 520)


In [191]:
### immediate Crosswalk for BBL and Property IDs Merging 
cw_join = property_ids[["property_id","bbl"]][~property_ids["bbl"].isnull()].drop_duplicates()
null_property_ids_enr1 = null_property_ids[[c for c in null_property_ids.columns if c !="property_id"]].merge(cw_join,how='left',on=["bbl"])
unresolved = null_property_ids_enr1[null_property_ids_enr1["property_id"].isnull()]
resolved = null_property_ids_enr1[~null_property_ids_enr1["property_id"].isnull()]
property_ids = pd.concat([property_ids,resolved]).drop_duplicates()

In [192]:
property_ids.groupby(["property_id"]).agg({"bbl":["nunique","count"],"source_api_url":"count"})

bbl       source_api_url
            nunique count          count
property_id                             
1407884.0         1     5              5
1443214.0         1     6              6
1732083.0         1     4              4
1847014.0         1     4              4
1944274.0         1     6              6
...             ...   ...            ...
35964199.0        0     0              1
35992350.0        0     0              1
36064989.0        1     1              1
36064990.0        1     1              1
36087754.0        1     1              1

[31189 rows x 3 columns]

(849, 520)
(5750, 520)


In [194]:
property_ids.groupby(["year_ending_year"]).agg({"property_id":["nunique","count"],"source_api_url":"count"})

property_id        source_api_url
                     nunique  count          count
year_ending_year                                  
2010                   11476  14307          14307
2011                   14874  17402          17402
2012                     804    804            804
2013                    9291   9405           9405
2014                    9416   9435           9435
2015                    6834   6908           6908
2016                    7059   7071           7071
2017                   11948  17491          17491
2018                   13378  13696          13696
2019                   13689  14367          14367
2020                   17205  17290          17290
2021                   13054  13164          13164
2022                   13401  13444          13444
2023                   14574  14803          14803

In [216]:
### Resolving Mulitple BBLS per property Id
test = property_ids[["property_id", "bbl", "address_1", "city","latitude", "longitude", "postcode", "year_ending_year"]].drop_duplicates()
test2 = (test.groupby("property_id").agg(bbl_nunique=("bbl", "nunique"),address_nunique=("address_1", "nunique"),
                                         year_min=("year_ending_year", "min"),year_max=("year_ending_year", "max"),).reset_index())
to_fix = test2[test2["bbl_nunique"] > 1]
to_fix = to_fix[((to_fix["year_min"]==2010)
        &(to_fix["year_max"]>=2017))]

for i, r in to_fix.iterrows():
    print(r["property_id"])
    temp_df = property_ids[["property_id","bbl","year_ending","address_1","city","postcode"]][property_ids["property_id"]==r['property_id']]
    print(temp_df['bbl'].unique())
    #Most frequent (mode) BBL for this property_id
    mode_bbl = temp_df['bbl'].mode(dropna=True).iloc[0]
    if ";" in mode_bbl:
        continue
    else:
        print(temp_df.shape)
        # Write that BBL back onto ALL rows for this property_id
        property_ids.loc[property_ids["property_id"] == r['property_id'], "bbl"] = mode_bbl

2506975.0
['2026140061' '2023140061']
(13, 6)
2639329.0
['2042520025' '2042520052']
(12, 6)
2640537.0
['4066780001' '4066770001']
(14, 6)
2657973.0
['1007537502' '1007530048']
(14, 6)
2658179.0
['1020647502' '1020640018']
(13, 6)
2658291.0
['2028610067' '2026800019']
(9, 6)
2658513.0
['1021347504' '1021340200']
(9, 6)
2658791.0
['1021807501' '1021800134']
(5, 6)
2658837.0
['1021807505' '1021800154']
(15, 6)
2661683.0
['3011907501' '3011900014']
(10, 6)
2664411.0
['4032587501' '4032580006']
(5, 6)
2664467.0
['4050407502' '4050400019']
(5, 6)
2682035.0
['1008357501' '1008350001']
(5, 6)
2686101.0
['4051230051' '4028640017']
(7, 6)
2691095.0
['2028879100' '2028879100; 2028870100']
(6, 6)
2693021.0
['4000627501' '4000627501;4000620046']
(5, 6)
2693044.0
['1012947501;1012940037' '1012940037']
(12, 6)
2702118.0
['1014387502' '1014380049']
(9, 6)
2704721.0
['3044147501' '3044157501']
(8, 6)
2708796.0
['1006430001' '1006430027']
(9, 6)
2711028.0
['2036230020' '2036230001']
(7, 6)
2715805.0
['3

In [218]:
### Putting together Processed Results So Far 
## Keepgin the unresolved with BBL values, dropping those that are null because BBL and Property ID are null.
print(unresolved[unresolved["bbl"].isnull()].shape)
print(unresolved[~unresolved["bbl"].isnull()].shape)
# test = pd.concat()
keep = unresolved[~unresolved["bbl"].isnull()]
keep["resolved_flag"]="UNRESOLVED"
property_ids["resolved_flag"]="RESOLVED"
residential3_refined= pd.concat([property_ids,keep])

(849, 521)
(5750, 521)


C:\Users\johnf\AppData\Local\Temp\ipykernel_62516\3801367619.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  keep["resolved_flag"]="UNRESOLVED"


#### Limiting by Years and for those Buildigns that have been in the Data for the years we want

In [224]:
buildings_2010_property_ids = residential3_refined[((residential3_refined["year_ending_year"].isin([2010,2011]))
                                                    &(~residential3_refined["property_id"].isnull()))]["property_id"].unique()
buildings_2010_bbls = residential3_refined[((residential3_refined["year_ending_year"].isin([2010,2011]))
                                            &(~residential3_refined["bbl"].isnull()))]["bbl"].unique()
print("CheckingGrouping - Prop ID")
print(residential3_refined[residential3_refined['property_id'].isin(buildings_2010_property_ids)].groupby(['year_ending_year']).agg({"address_1":"count"}).reset_index())
print("CheckingGrouping - BBL")
print(residential3_refined[residential3_refined['bbl'].isin(buildings_2010_bbls)].groupby(['year_ending_year']).agg({"address_1":"count"}).reset_index())
### Limtiing to 2017 and before 
residential3_refined_b4_2018 = residential3_refined[residential3_refined["year_ending_year"]<2018]
print("YEARS")
print(residential3_refined_b4_2018['year_ending_year'].unique())
residential3_2010_2017 = residential3_refined_b4_2018[((residential3_refined_b4_2018['property_id'].isin(buildings_2010_property_ids))
                                                       |(residential3_refined_b4_2018['bbl'].isin(buildings_2010_bbls)))]



CheckingGrouping - Prop ID
    year_ending_year  address_1
0               2010      14307
1               2011      17369
2               2012        562
3               2013       8191
4               2014       8183
5               2015       5872
6               2016       5839
7               2017       8526
8               2018       6283
9               2019       6426
10              2020       8372
11              2021       5707
12              2022       5820
13              2023       6591
CheckingGrouping - BBL
    year_ending_year  address_1
0               2010      16573
1               2011      20773
2               2012        540
3               2013       8097
4               2014       8118
5               2015       5848
6               2016       5831
7               2017       8468
8               2018       6270
9               2019       6413
10              2020       8359
11              2021       5698
12              2022       5807
13              2023  

In [226]:
### Building Info Only 
building_info =residential3_2010_2017[["property_id",'bbl',"address_1","address_2","city","postcode",'county','borough',"latitude","longitude"]].drop_duplicates()
# building_info.to_csv("building_info.csv", index=False)

In [51]:
# building_info = pd.read_csv("building_info.csv")

#### Picking up here while the secondary API code runs to ensure we're pulling all years

In [227]:
# NYC biasing bounds (rough NYC box)
NYC_BOUNDS = {
    "southwest": {"lat": 40.477399, "lng": -74.259090},
    "northeast": {"lat": 40.917577, "lng": -73.700272},
}

# Common misspellings / normalizations seen in your data
COMMON_FIXES = {
    r"\bWASHIGNTON\b": "WASHINGTON",
    r"\bWASHNGTN\b": "WASHINGTON",
    r"\bALBERMALE\b": "ALBEMARLE",
    r"\bALBERMALE\b": "ALBEMARLE",
    r"\bAMSETRDAM\b": "AMSTERDAM",
    r"\bKINGBRIDGE\b": "KINGSBRIDGE",
    r"\bOVINGTON\b": "OVINGTON",   # keep as is (looks fine)
    r"\bNAGLE HOUSE\b": "NAGLE AVE",  # likely street not building name
    r"\bNAGLE\b": "NAGLE",  # pass-through
    r"\bST\s*NICOLAS\b": "ST NICHOLAS",
    r"\bOILVER\b": "OLIVER",
    r"\bRIVER SIDE\b": "RIVERSIDE",
    r"\bSTEET\b": "STREET",
    r"\bSREET\b": "STREET",
    r"\bFOR WASHINGTON\b": "FORT WASHINGTON",
    r"\bFT\.?\b": "FORT",
    r"\bBIVONA\b": "BIVONA",  # leave
    r"\bCLARKE\b": "CLARKE",  # leave
    r"\bCAROLL\b": "CARROLL",
    r"\bCLAFFIN\b": "CLAFLIN",
    r"\bDEKALB\b": "DEKALB",
    r"\bLAFAYETTE\b": "LAFAYETTE",
    r"\bW\s*MOSHOLU\b": "W MOSHOLU",
    r"\bE\s*MOSHOLU\b": "E MOSHOLU",
    r"\bMOSHOLU\b": "MOSHOLU",
    r"\bPARSON\b": "PARSONS",
    r"\bPARSONS BLVD?\b": "PARSONS BLVD",
}

# Street type vocabulary to help "best-guess" if clearly missing
STREET_TYPES = ["ST", "AVE", "RD", "BLVD", "PL", "LN", "DR", "CT", "PKWY", "TER", "PLZ"]

def _norm_ws(s: Optional[str]) -> Optional[str]:
    if s is None or (isinstance(s, float) and pd.isna(s)):
        return None
    s = str(s)
    # HTML escapes commonly seen
    s = s.replace("&amp;", "&")
    s = s.replace("\u2013", "-").replace("\u2014", "-")
    s = s.strip()
    s = re.sub(r"\s+", " ", s)
    return s if s else None

def _apply_common_fixes(u: str) -> str:
    for pat, repl in COMMON_FIXES.items():
        u = re.sub(pat, repl, u, flags=re.IGNORECASE)
    return u

def _std_unit(s: Optional[str]) -> Optional[str]:
    u = _norm_ws(s)
    if not u:
        return None
    u = u.upper().replace("#", "").strip()
    if not u:
        return None
    u = re.sub(r"^(APARTMENT|APT\.?)\s*", "APT ", u)
    u = re.sub(r"^(SUITE|STE\.?)\s*", "STE ", u)
    u = re.sub(r"^(FLOOR|FL\.?)\s*", "FL ", u)
    if not re.match(r"^(APT|STE|FL)\b", u):  # bare "5B" → "APT 5B"
        u = f"APT {u}"
    return u

def _std_street(u: Optional[str]) -> Optional[str]:
    u = _norm_ws(u)
    if not u:
        return None
    u = _apply_common_fixes(u.upper())

    repl = {
        " STREET": " ST",
        " AVENUE": " AVE",
        " ROAD": " RD",
        " BOULEVARD": " BLVD",
        " PLACE": " PL",
        " LANE": " LN",
        " DRIVE": " DR",
        " COURT": " CT",
        " PARKWAY": " PKWY",
        " TERRACE": " TER",
        " PLAZA": " PLZ",
    }
    for k, v in repl.items():
        u = re.sub(k + r"\b", v, u)

    # Normalize ordinals and add missing "TH/ST/ND/RD" on bare numbers like "E 48" or "West104"
    u = re.sub(r"\b(\d+)\s*(ST|ND|RD|TH)\b", lambda m: f"{int(m.group(1))}{m.group(2)}", u)
    u = re.sub(r"\b(WEST|W|EAST|E)\s*(\d{1,3})(?=\b)", r"\1 \2", u)  # ensure space
    def add_ordinal(m):
        n = int(m.group(2))
        suf = "TH"
        if n % 10 == 1 and n % 100 != 11: suf = "ST"
        elif n % 10 == 2 and n % 100 != 12: suf = "ND"
        elif n % 10 == 3 and n % 100 != 13: suf = "RD"
        return f"{m.group(1)} {n}{suf}"
    u = re.sub(r"\b(WEST|W|EAST|E)\s+(\d{1,3})\b(?!\s*(ST|AVE|RD|BLVD|PL|LN|DR|CT|PKWY|TER|PLZ))", add_ordinal, u)

    # Collapse multiple addresses to last one here (we also split earlier in the pipeline)
    if "/" in u:
        u = u.split("/")[-1].strip()

    # Queens: hyphenated house numbers like "41-07 42ND ST" are valid; leave them as-is
    # Best-guess add a street type if line is like "308 WEST 104TH" (no type) or "160 E 48TH"
    if re.search(r"\b(WEST|W|EAST|E|NORTH|N|SOUTH|S)\b", u) and re.search(r"\b\d{1,3}(ST|ND|RD|TH)\b", u) and not re.search(r"\b(ST|AVE|RD|BLVD|PL|LN|DR|CT|PKWY|TER|PLZ)\b", u):
        u = u + " ST"

    # Convert key phrases
    u = re.sub(r"\bFT\b", "FORT", u)

    return u.title()

def _borough_from_fields(borough, county, city) -> Optional[str]:
    vals = " ".join([str(x) for x in [borough, county, city] if pd.notna(x)]).upper()
    if any(x in vals for x in ["MANHATTAN", "NEW YORK", "NY COUNTY"]): return "MANHATTAN"
    if "BRONX" in vals: return "BRONX"
    if any(x in vals for x in ["BROOKLYN", "KINGS"]): return "BROOKLYN"
    if "QUEENS" in vals: return "QUEENS"
    if any(x in vals for x in ["STATEN", "RICHMOND"]): return "STATEN ISLAND"
    return None

def _usps_city_from_borough(city, borough_norm) -> Optional[str]:
    c = (_norm_ws(city) or "").upper()
    if borough_norm == "MANHATTAN": return "New York"
    if borough_norm == "BRONX": return "Bronx"
    if borough_norm == "BROOKLYN": return "Brooklyn"
    if borough_norm == "QUEENS":
        # USPS accepts neighborhoods, but "Queens" is safest default
        return "Queens" if c in ["", "QUEENS", "NAN"] else c.title()
    if borough_norm == "STATEN ISLAND": return "Staten Island"
    return c.title() if c else None

def _std_zip5(z):
    if z is None or (isinstance(z, float) and pd.isna(z)):
        return None
    s = re.sub(r"\D", "", str(z))
    return s[:5] if len(s) >= 5 else None

# ---------- NEW: address candidate generator ----------

SEP_PATTERN = re.compile(r"\s*(?:;|,|/|&{1,2}| and | AND |\bETAL\b|\bet al\b|\(|\)|\bAKA\b|\bA\/K\/A\b)\s*", flags=re.IGNORECASE)

def _is_pobox(s: str) -> bool:
    return bool(re.search(r"\bP\.?\s*O\.?\s*BOX\b", s, flags=re.IGNORECASE))

def _has_street_name(u: str) -> bool:
    # needs something besides just a house number or zeros
    return bool(re.search(r"\b[A-Z][A-Z]+\b", u)) and not re.fullmatch(r"\d+(-\d+)?", re.sub(r"[^0-9\-]", "", u or ""))

def _expand_number_ranges(u: str) -> List[str]:
    """
    '2078-84-90 Morris Ave' -> ['2078 Morris Ave','2084 Morris Ave','2090 Morris Ave']
    '1051-1057-1061-1065 Boston Rd' -> same expansion
    """
    m = re.search(r"\b(\d{1,6}(?:-\d{1,6})+)\s+([A-Z].+)$", u)
    if not m:
        return [u]
    block = m.group(1)
    tail = m.group(2)
    parts = [p for p in re.split(r"-", block) if p]
    base = parts[0]
    expanded = []
    for p in parts:
        if len(p) < len(base):  # "2078-84" -> fill high-order digits from base
            p = base[:len(base)-len(p)] + p
        expanded.append(f"{p} {tail}")
    return expanded

def _split_multi(s: str) -> List[str]:
    s = SEP_PATTERN.sub(" | ", s)  # unify to pipe, then split
    tokens = [t.strip(" .") for t in s.split("|") if _norm_ws(t)]
    # Remove address-like fragments that are obviously units/notes only
    return [t for t in tokens if t and not re.match(r"^(APT|STE|FL|B#\d+)\b", t, flags=re.IGNORECASE)]

def build_candidates(row) -> List[Tuple[str, dict, str]]:
    """
    Returns list of (query, components, reason_tag)
    """
    a1 = _std_street(row.get("address_line1_clean"))
    a2 = _std_unit(row.get("address_line2_clean"))
    borough = _borough_from_fields(row.get("borough_clean"), row.get("county"), row.get("city_clean"))
    city = _usps_city_from_borough(row.get("city_clean"), borough)
    zip5 = _std_zip5(row.get("zip"))
    state = STATE_ABBR

    raw = _norm_ws(row.get("geocode_key") or row.get("address_1") or "")
    raw = _apply_common_fixes(raw.upper()) if raw else ""
    raw_parts = _split_multi(raw) if raw else []

    # Build an address seed from clean fields
    seeds = []
    if a1:
        seeds.append(a1)
    seeds.extend(_expand_number_ranges(x) for x in raw_parts)
    seeds = [y for x in seeds for y in (x if isinstance(x, list) else [x])]
    seeds = [s for s in seeds if s] or ([raw.title()] if raw else [])

    candidates = []
    for s in seeds:
        s_norm = _std_street(s)
        if not s_norm:
            continue
        if _is_pobox(s_norm):
            candidates.append((None, {}, "POBOX"))
            continue
        if not _has_street_name(s_norm):
            candidates.append((None, {}, "MISSING_STREET"))
            continue

        line = s_norm
        parts = [line]
        # Append unit if it looks like an apartment (rarely helpful for geocode but OK)
        # if a2: parts.append(a2)

        locality = city or (borough.title() if borough else None)
        tail = ", ".join([p for p in [locality, state, zip5] if p])
        query = f"{' '.join(parts)}, {tail}" if tail else " ".join(parts)

        comps = {"administrative_area": state}
        if locality: comps["locality"] = locality

        candidates.append((query, comps, "OK"))
    # Deduplicate, keep order
    seen = set()
    out = []
    for q, c, r in candidates:
        key = (q or r, tuple(sorted(c.items())))
        if key not in seen:
            seen.add(key)
            out.append((q, c, r))
    return out


In [228]:
# Cleaned drafts
building_info["address_line1_clean"] = building_info["address_1"].apply(_std_street)
building_info["address_line2_clean"] = building_info["address_2"].apply(_std_unit)

building_info["borough_clean"] = [
    _borough_from_fields(b, c, ci)
    for b, c, ci in zip(building_info.get("borough"), building_info.get("county"), building_info.get("city"))
]
building_info["city_clean"] = [
    _usps_city_from_borough(ci, bn)
    for ci, bn in zip(building_info.get("city"), building_info["borough_clean"])
]
building_info["state_clean"] = STATE_ABBR
building_info["postal_code_5_clean"] = building_info.get("postal_code").apply(_std_zip5) if "postal_code" in building_info.columns else None

# Mark rows missing coords
building_info["needs_geocoding"] = building_info["latitude"].isna() | building_info["longitude"].isna()

# a stable address key to dedupe geocoding calls
def addr_key(r):
    parts = [
        _norm_ws(r["address_line1_clean"]) or "",
        _norm_ws(r["address_line2_clean"]) or "",
        _norm_ws(r["city_clean"]) or "",
        STATE_ABBR,
        _norm_ws(r["postal_code_5_clean"]) or "",
    ]
    return "|".join(parts).upper()

building_info["geocode_key"] = building_info.apply(addr_key, axis=1)

building_info.head(3)[[
    "property_id",'bbl',"address_1","address_2","city","borough","county",
    "address_line1_clean","address_line2_clean","city_clean","state_clean","postal_code_5_clean",
    "needs_geocoding","geocode_key"
]]


,property_id,bbl,address_1,address_2,city,borough,county,address_line1_clean,address_line2_clean,city_clean,state_clean,postal_code_5_clean,needs_geocoding,geocode_key
86767,2638790.0,4001610033,43-22,NaN,Sunnyside,NaN,NaN,43-22,None,Sunnyside,NY,None,True,43-22||SUNNYSIDE|NY|
86768,5841233.0,2024610111,1011 Carroll Place,NaN,Bronx,BRONX,NaN,1011 Carroll Pl,None,Bronx,NY,None,False,1011 CARROLL PL||BRONX|NY|
86769,2707907.0,3074560006,2626 Homecrest Avenue,NaN,Brooklyn,BROOKLYN,NaN,2626 Homecrest Ave,None,Brooklyn,NY,None,False,2626 HOMECREST AVE||BROOKLYN|NY|


In [230]:
print(building_info.shape)
building_info = building_info.dropna(subset=["address_1","address_2","city","postcode","county","borough",
                                             "latitude","longitude","address_line1_clean"],how='all')
print(building_info.shape)

(37998, 18)
(37998, 18)


In [244]:
### Geocode The Null lat long 
with_coord = building_info[building_info["needs_geocoding"]==False]
print(with_coord.shape)
without_coord = building_info[building_info["needs_geocoding"]==True]
print(without_coord.shape)

(35919, 18)
(2079, 18)


In [245]:
without_coord['city'][without_coord['city']=='Flushinig']='Flushing'
without_coord['borough_clean'][(without_coord['city'].isin(['Sunnyside', 'Astoria', 'Jackson Heights', 'Flushing', 'Rego Park',
                                                   'Forest Hills', 'Oakland Gardens', 'Bayside', 'Kew Gardens','flushing',
                                                   'Woodside', 'Flushinig', 'Larchmont', 'Ridgewood','Jamaica', 'Albertson'])
                                &(without_coord['borough_clean'].isnull()))]='QUEENS'
### Manual Fixes where possible 
without_coord["borough_clean"][((without_coord['borough_clean'].isnull())&
               (without_coord["address_1"]=="3115 brighton 6th"))]="BROOKLYN"
without_coord["borough_clean"][((without_coord['borough_clean'].isnull())&
               (without_coord["address_1"]=="144-35/39 Sanford Avenue"))]="QUEENS"
without_coord["borough_clean"][((without_coord['borough_clean'].isnull())&
               (without_coord["address_1"]=="183-11 Hillside Ave"))]="QUEENS"
without_coord["borough_clean"][((without_coord['borough_clean'].isnull())&
               (without_coord["address_1"]=="103-30/26 68th Ave"))]="QUEENS"
without_coord["borough_clean"][((without_coord['borough_clean'].isnull())&
               (without_coord["address_1"]=="71-11 -71-23 162ND STREET"))]="QUEENS"
without_coord = without_coord[~without_coord['borough_clean'].isnull()]
print(without_coord.shape)

(2058, 18)


C:\Users\johnf\AppData\Local\Temp\ipykernel_62516\277626894.py:1: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  without_coord['city'][without_coord['city']=='Flushinig']='Flushing'
C:\Users\johnf\AppData\Local\Temp\ipykernel_62516\277626894.

### Start of Geocoding 

In [ ]:
# reading in past resu;ts
# results_df = pd.concat(results)
# results_df.to_csv("results_geo.csv",index=False)
# results_df = pd.read_csv("results_geo.csv")
# failed_df = pd.concat(failed)
# failed_df.to_csv("failed_geo.csv",index=False)
# failed_df = pd.read_csv("failed_geo.csv")

In [243]:
# merging_cols = [c for c in without_coord.dropna(how='all',axis=1).columns if c not in ["latitude","longitude"]]
# without_coord_enr = without_coord[merging_cols].merge(building_info_geocoded,
#                                   on=[i for i in merging_cols if i in building_info_geocoded.columns],
#                                   how='left')
# done_already=without_coord_enr[((~without_coord_enr["latitude"].isnull()) & (~without_coord_enr["longitude"].isnull()))]
# print(done_already.shape)
# without_coord = without_coord_enr[((without_coord_enr["latitude"].isnull()) | (without_coord_enr["longitude"].isnull()))]
# print(without_coord.shape)

(0, 22)
(2058, 22)


In [249]:
# ---- GEOCODING LOOP (with retries/candidates/NYC bias) ----
results, failed = [], []

def geocode_row(row):
    cands = build_candidates(row)
    # Always add *raw borough + state* fallback for named complexes (e.g., NYCHA) to try to at least anchor
    borough = _borough_from_fields(row.get("borough_clean"), row.get("county"), row.get("city_clean"))
    if borough:
        cands.append((f"{borough.title()}, NY", {"administrative_area": "NY", "locality": borough.title()}, "BOROUGH_ONLY"))

    for query, comps, tag in cands:
        if not query:
            continue
        try:
            resp = gmaps.geocode(
                query,
                region="us",
                components=comps,
                bounds=NYC_BOUNDS
            )
        except Exception as e:
            print("Geocode error:", e)
            resp = []

        if resp:
            # prefer rooftop or range_interpolated results
            best = sorted(resp, key=lambda r: {"ROOFTOP":0, "RANGE_INTERPOLATED":1}.get(r.get("geometry",{}).get("location_type",""), 2))[0]
            loc = best["geometry"]["location"]
            return loc["lat"], loc["lng"], tag, best.get("formatted_address"), best.get("place_id")

    return None, None, "NO_HIT", None, None

# assumes without_coord is a DataFrame with the relevant columns
for i, r in without_coord[["property_id",'bbl',"geocode_key","address_line1_clean","address_line2_clean","borough_clean","city_clean","county"]].drop_duplicates().iterrows():
    # keep your i >= guard if needed
    temp_df = pd.DataFrame([r])
    lat, lng, tag, faddr, pid = geocode_row(r)

    if lat is not None and lng is not None:
        temp_df["latitude"]  = lat
        temp_df["longitude"] = lng
        temp_df["geocode_tag"] = tag
        temp_df["formatted_address"] = faddr
        temp_df["place_id"] = pid
        results.append(temp_df)
    else:
        temp_df["geocode_tag"] = tag  # e.g., POBOX, MISSING_STREET, NO_HIT
        failed.append(temp_df)


In [252]:
print(len(results))
print(len(failed)) ## Zero failures

2055
0


In [254]:
# reading in past resu;ts
results_df = pd.concat(results)
# results_df.to_csv("results_geo.csv",index=False)

In [265]:
### Property ID Present Without_coord
print(without_coord.shape)
without_coord_geo = without_coord.dropna(how='all',axis=1).merge(results_df, 
                                                                 on=["property_id",'bbl',"geocode_key","address_line1_clean","address_line2_clean","borough_clean","city_clean","county"],
                                                                 how='left')

without_coord_geo[without_coord_geo['latitude'].isnull()]


(2058, 18)


,property_id,bbl,address_1,address_2,city,postcode,county,borough,address_line1_clean,address_line2_clean,borough_clean,city_clean,state_clean,needs_geocoding,geocode_key,latitude,longitude,geocode_tag,formatted_address,place_id


In [270]:
building_info_geocoded= pd.concat([with_coord,without_coord_geo])
# building_info_geocoded.to_csv("building_info_geocoded.csv",index=False)
# building_info_geocoded = pd.read_csv("building_info_geocoded.csv")

In [272]:
building_info_geocoded["geocode_tag"].unique()

array([nan, 'BOROUGH_ONLY'], dtype=object)

In [273]:
### Limit the resutlsd based on Building-specific Lat Long, not borough centroids.
building_info_geocoded = building_info_geocoded[building_info_geocoded['geocode_tag'].isnull()]
### NEEED TO ADD THE BBL BY ADDRESS.(Where needed)
print(building_info_geocoded.shape)

(35919, 21)


In [280]:
need_bbl = building_info_geocoded[((building_info_geocoded['bbl'].isnull())
                                   |(building_info_geocoded['bbl'].astype(str).str.contains(";")))]
print(need_bbl.shape)
has_bbl = building_info_geocoded[((~building_info_geocoded['bbl'].isnull())
                                  &(~building_info_geocoded['bbl'].astype(str).str.contains(";")))]
print(has_bbl.shape)
# list(has_bbl['bbl'].unique())

(123, 21)
(35796, 21)


['2024610111',
 '3074560006',
 '1010710042',
 '4012750045',
 '4012720012',
 '4058630002',
 '4058650040',
 '4058630200',
 '1008060026',
 '1005640031',
 '1018510005',
 '1012440072',
 '3063440033',
 '1018680001',
 '2025160058',
 '4014337501',
 '2042840020',
 '4058650030',
 '4058570110',
 '4012680033',
 '4014200001',
 '1008480061',
 '1008867502',
 '3086880035',
 '1012720001',
 '1014460023',
 '3078190046',
 '3075590011',
 '3050650030',
 '4066980040',
 '1010080026',
 '1011860025',
 '1021720060',
 '1008030007',
 '4014890018',
 '1018420029',
 '4012630015',
 '1008830044',
 '1014600047',
 '1008020022',
 '1019940038',
 '1010897501',
 '2025050064',
 '2028780120',
 '3012970006',
 '3067300001',
 '2043190015',
 '4053740023',
 '2026770070',
 '3051240040',
 '2059040267',
 '3067140041',
 '3067220001',
 '2045500012',
 '3011890024',
 '2033140041',
 '1014100033',
 '2032490006',
 '2032490046',
 '1004270002',
 '1016210014',
 '1003450048',
 '3014220064',
 '1010800010',
 '3056640035',
 '4067110027',
 '10195300

### Enriching the Geocoded Building Information with BBLs using another API 

In [281]:
df = need_bbl.copy()
rows = []

with requests.Session() as s:
    for i, r in df.iterrows():
        addr = str(r.get("address_line1_clean", "")).strip()
        boro = str(r.get("borough_clean", "")).strip()
        if not addr:
            rows.append((i, None, None, None, None, None)); continue

        q = f"{addr}, {boro}, NY" if boro else f"{addr}, NY"
        try:
            resp = s.get("https://geosearch.planninglabs.nyc/v2/search",
                         params={"text": q, "size": 5}, timeout=20)
            feats = resp.json().get("features", [])
            # pick the best feature: prefer layer=='address' and same borough; else highest confidence
            best, best_score = None, (-1, -1.0)
            for f in feats:
                p = f.get("properties", {})
                layer = p.get("layer")
                conf  = float(p.get("confidence", 0) or 0)
                fb    = (p.get("borough") or ((p.get("addendum") or {}).get("pad") or {}).get("boroughName") or "")
                score = (2 if layer == "address" else 0) + (1 if boro and boro.upper() in str(fb).upper() else 0)
                if (score, conf) > best_score:
                    best, best_score = f, (score, conf)

            if best:
                props = best.get("properties", {})
                add   = (props.get("addendum") or {}).get("pad") or {}
                bbl   = add.get("bbl") or props.get("bbl")
                bin_  = add.get("bin") or props.get("bin")

                # normalize BBL to a 10-digit string
                if isinstance(bbl, str):
                    digits = re.sub(r"\D", "", bbl)
                    bbl = digits.zfill(10) if digits else None
                elif bbl is not None:
                    try: bbl = str(int(bbl)).zfill(10)
                    except: bbl = None

                rows.append((i, bbl, bin_, props.get("confidence"), props.get("label") or props.get("name"), q))
            else:
                rows.append((i, None, None, None, None, q))

        except Exception as e:
            rows.append((i, None, None, None, f"ERR:{type(e).__name__}", q))

        time.sleep(0.1)  # tiny throttle to be polite

# assemble + join back
out = pd.DataFrame(rows, columns=["__idx","BBL_from_api","BIN_from_api","confidence","match_label","query"]).set_index("__idx")

In [292]:
## Putting all tegehter and cleaning 
need_bbl_enr = need_bbl.join(out)
building_info_geocoded= pd.concat([need_bbl_enr, has_bbl])
# building_info_geocoded.to_csv("BuildingInfoGeocoded_GEO_BBL.csv",index=False)

In [299]:
## Cleaning up some
building_info_geocoded['bbl'] = building_info_geocoded['BBL_from_api'].combine_first(building_info_geocoded['bbl'])
building_info_geocoded = building_info_geocoded.drop(columns=["BBL_from_api","BIN_from_api","confidence"])
building_info_geocoded = building_info_geocoded.dropna(how='all',axis=1)

In [301]:
building_info_geocoded

,property_id,bbl,address_1,address_2,city,postcode,county,borough,latitude,longitude,address_line1_clean,address_line2_clean,borough_clean,city_clean,state_clean,needs_geocoding,geocode_key,match_label,query
86885,2833318.0,1009350005,530 Second Avenue,NaN,New York,10016,NaN,MANHATTAN,40.741923,-73.977933,530 Second Ave,None,MANHATTAN,New York,NY,False,530 SECOND AVE||NEW YORK|NY|,"530 SECOND AVENUE, New York, NY, USA","530 Second Ave, MANHATTAN, NY"
87266,6260224.0,3053070045,70 Dahill Road,NaN,Brooklyn,11218,NaN,BROOKLYN,40.645638,-73.981046,70 Dahill Rd,None,BROOKLYN,Brooklyn,NY,False,70 DAHILL RD||BROOKLYN|NY|,"70 DAHILL ROAD, Brooklyn, NY, USA","70 Dahill Rd, BROOKLYN, NY"
87267,6260314.0,1020910048,676 Riverside Drive,NaN,New York,10031,NaN,MANHATTAN,40.826851,-73.952170,676 Riverside Dr,None,MANHATTAN,New York,NY,False,676 RIVERSIDE DR||NEW YORK|NY|,"676 RIVERSIDE DRIVE, New York, NY, USA","676 Riverside Dr, MANHATTAN, NY"
87855,6272091.0,1021700292,320 Wadsworth Ave,NaN,NY,10040,NaN,MANHATTAN,40.854999,-73.931202,320 Wadsworth Ave,None,MANHATTAN,New York,NY,False,320 WADSWORTH AVE||NEW YORK|NY|,"320 WADSWORTH AVENUE, New York, NY, USA","320 Wadsworth Ave, MANHATTAN, NY"
87876,2810721.0,3060830041,9108 Colonial Road,NaN,Brooklyn,11209,NaN,BROOKLYN,40.621136,-74.037902,9108 Colonial Rd,None,BROOKLYN,Brooklyn,NY,False,9108 COLONIAL RD||BROOKLYN|NY|,"9108 COLONIAL ROAD, Brooklyn, NY, USA","9108 Colonial Rd, BROOKLYN, NY"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38131,NaN,2036530001,1560 Bruckner Blvd,NaN,Bronx,10473,BRONX,BRONX,40.823685,-73.875968,1560 Bruckner Blvd,None,BRONX,Bronx,NY,False,1560 BRUCKNER BLVD||BRONX|NY|,NaN,NaN
38138,NaN,2040420350,1610 Matthews Avenue,NaN,Bronx,10462,BRONX,BRONX,40.844451,-73.861078,1610 Matthews Ave,None,BRONX,Bronx,NY,False,1610 MATTHEWS AVE||BRONX|NY|,NaN,NaN
38219,NaN,1008460060,867 Broadway,NaN,New York,10003,NEW YORK,MANHATTAN,40.737567,-73.990207,867 Broadway,None,MANHATTAN,New York,NY,False,867 BROADWAY||NEW YORK|NY|,NaN,NaN
38306,NaN,1010260041,210 W 55th St,NaN,New York,10019,NEW YORK,MANHATTAN,40.764491,-73.981651,210 W 55Th St,None,MANHATTAN,New York,NY,False,210 W 55TH ST||NEW YORK|NY|,NaN,NaN


### Pulling in Pluto Data for Num Floors and Height etc`

In [304]:
# SOURCE # https://www.nyc.gov/content/planning/pages/resources/datasets/mappluto-pluto-change#mappluto
# ===================== CONFIG =====================
SRC   = r"C:/Users/johnf/Downloads/nyc_mappluto_25v3_fgdb/MapPLUTO25v3.gdb"
LAYER = "MapPLUTO_25v3_clipped"

# PLUTO columns you need (no geometry necessary)
PLUTO_COLS = ["BBL","LandUse","NumFloors","UnitsRes","BldgClass","Block","Borough","BoroCode"]

# How many distinct BLOCKS per chunk (per borough)
BLOCKS_PER_CHUNK = 2000  # tune: 1000–4000 are typical sweet spots

# ===================== HELPERS =====================
def norm_bbl_series(s: pd.Series) -> pd.Series:
    """Normalize to 10-digit BBL strings; non-digits removed; blanks -> NA."""
    s = s.astype(str).str.replace(r"\D", "", regex=True).str.zfill(10)
    return s.mask(s.eq("0000000000"))

def boro_code_from_bbl_str(bbl: str) -> int:
    """First digit of BBL is borough code 1..5."""
    return int(bbl[0]) if isinstance(bbl, str) and bbl and bbl[0].isdigit() else None

def block_from_bbl_str(bbl: str) -> int:
    """Digits 2..6 (5 digits) are block."""
    try:
        return int(bbl[1:6])
    except Exception:
        return None

def pull_where(where: str, cols: list[str]) -> pd.DataFrame:
    """Run filtered attribute read; returns pandas.DataFrame (no geometry)."""
    return pyogrio.read_dataframe(
        SRC, layer=LAYER,
        columns=[c for c in cols if c != "geometry"],
        read_geometry=False,
        where=where,
    )

[prep] Unique LL84 BBLs: 12,028


In [339]:
# SOURCE # https://www.nyc.gov/content/planning/pages/resources/datasets/mappluto-pluto-change#mappluto
# ===================== CONFIG =====================
SRC   = r"C:/Users/johnf/Downloads/nyc_mappluto_25v3_fgdb/MapPLUTO25v3.gdb"
LAYER = "MapPLUTO_25v3_clipped"

# PLUTO columns you need (no geometry necessary)
PLUTO_COLS = ["BBL","LandUse","NumFloors","UnitsRes","BldgClass","Block","Borough","BoroCode"]

# How many distinct BLOCKS per chunk (per borough)
BLOCKS_PER_CHUNK = 2000  # tune: 1000–4000 are typical sweet spots

# ===================== HELPERS =====================
def norm_bbl_series(s: pd.Series) -> pd.Series:
    """Normalize to 10-digit BBL strings; handle '1234567890.0' correctly."""
    # Convert to string & trim
    s = s.astype(str).str.strip()

    # If it looks like 'digits.0' or 'digits.00', drop the decimal part
    s = s.str.replace(r"\.0+$", "", regex=True)

    # Remove any remaining non-digits, then pad to 10
    s = s.str.replace(r"\D", "", regex=True).str.zfill(10)

    # Turn "0000000000" into NA
    return s.mask(s.eq("0000000000"))

def boro_code_from_bbl_str(bbl: str) -> int:
    """First digit of BBL is borough code 1..5."""
    return int(bbl[0]) if isinstance(bbl, str) and bbl and bbl[0].isdigit() else None

def block_from_bbl_str(bbl: str) -> int:
    """Digits 2..6 (5 digits) are block."""
    try:
        return int(bbl[1:6])
    except Exception:
        return None

def pull_where(where: str, cols: list[str]) -> pd.DataFrame:
    """Run filtered attribute read; returns pandas.DataFrame (no geometry)."""
    return pyogrio.read_dataframe(
        SRC, layer=LAYER,
        columns=[c for c in cols if c != "geometry"],
        read_geometry=False,
        where=where,
    )

In [340]:
# ===================== 1) LL84 BBL PREP =====================
ll84 = building_info_geocoded.copy()
ll84["bbl"] = norm_bbl_series(ll84["bbl"])
ll84 = ll84[ll84["bbl"].notna()].copy()

bbls = ll84["bbl"].unique().tolist()
print(f"[prep] Unique LL84 BBLs: {len(bbls):,}")

# ===================== 2) PULL PLUTO BY BBL IN(...) BATCHES =====================
parts = []
t0 = time.time()

BATCH = 800   # tune if needed
total_chunks = math.ceil(len(bbls) / BATCH)
print(f"[plan] {total_chunks} PLUTO chunks via BBL IN(...) batches")

for ci in range(total_chunks):
    chunk = bbls[ci*BATCH:(ci+1)*BATCH]

    # We already know "BBL = '...'" works, so quote as text
    in_list = ",".join(f"'{bb}'" for bb in chunk)
    where = f"BBL IN ({in_list})"

    part = pull_where(where, PLUTO_COLS)
    print(
        f"[pull] chunk {ci+1}/{total_chunks} | requested {len(chunk):,} BBLs "
        f"| pulled {len(part):,}"
    )

    if not part.empty:
        part["BBL"] = norm_bbl_series(part["BBL"])
        parts.append(part)

elapsed = time.time() - t0
pluto_by_bbl = (
    pd.concat(parts, ignore_index=True) if parts
    else pd.DataFrame(columns=PLUTO_COLS)
)

print(f"[done] Kept PLUTO rows: {len(pluto_by_bbl):,} in {elapsed:,.1f}s")

# ===================== 3) MERGE BACK TO LL84 + FILTER UNDER 5 STORIES =====================
ll84_enriched = ll84.merge(
    pluto_by_bbl.drop_duplicates("BBL"),
    left_on="bbl",
    right_on="BBL",
    how="left"
)

ll84_enriched["stories"] = ll84_enriched["NumFloors"]

res_codes = {"01", "02", "03", "04"}
is_res = (
    ll84_enriched["LandUse"].astype(str).isin(res_codes)
    | (ll84_enriched["UnitsRes"].fillna(0) > 0)
    | ll84_enriched.get(
        "BldgClass",
        pd.Series(index=ll84_enriched.index, dtype="object")
      ).fillna("").str.startswith(("A", "B"))
)

under5 = ll84_enriched[
    is_res
    & ll84_enriched["stories"].notna()
    & (ll84_enriched["stories"] < 5)
].copy()

print(
    f"[summary] LL84 rows: {len(ll84_enriched):,} | "
    f"Residential <5 stories: {len(under5):,}"
)


[prep] Unique LL84 BBLs: 12,028
[plan] 16 PLUTO chunks via BBL IN(...) batches
[pull] chunk 1/16 | requested 800 BBLs | pulled 793
[pull] chunk 2/16 | requested 800 BBLs | pulled 791
[pull] chunk 3/16 | requested 800 BBLs | pulled 796
[pull] chunk 4/16 | requested 800 BBLs | pulled 800
[pull] chunk 5/16 | requested 800 BBLs | pulled 798
[pull] chunk 6/16 | requested 800 BBLs | pulled 798
[pull] chunk 7/16 | requested 800 BBLs | pulled 795
[pull] chunk 8/16 | requested 800 BBLs | pulled 777
[pull] chunk 9/16 | requested 800 BBLs | pulled 791
[pull] chunk 10/16 | requested 800 BBLs | pulled 782
[pull] chunk 11/16 | requested 800 BBLs | pulled 761
[pull] chunk 12/16 | requested 800 BBLs | pulled 715
[pull] chunk 13/16 | requested 800 BBLs | pulled 687
[pull] chunk 14/16 | requested 800 BBLs | pulled 686
[pull] chunk 15/16 | requested 800 BBLs | pulled 749
[pull] chunk 16/16 | requested 28 BBLs | pulled 28
[done] Kept PLUTO rows: 11,547 in 349.7s
[summary] LL84 rows: 35,918 | Residential <

# ---------------------------------

In [347]:
# ll84_enriched.to_csv("ll84_enriched_20251108.csv",index=False)

In [345]:
print(ll84_enriched[ll84_enriched["NumFloors"].isnull()].shape)
# ll84_enriched[~ll84_enriched["NumFloors"].isnull()]

(1103, 28)


In [348]:
ll84_enriched

,property_id,bbl,address_1,address_2,city,postcode,county,borough,latitude,longitude,address_line1_clean,address_line2_clean,borough_clean,city_clean,state_clean,needs_geocoding,geocode_key,match_label,query,BBL,BldgClass,Block,BoroCode,Borough,LandUse,NumFloors,UnitsRes,stories
0,2833318.0,1009350005,530 Second Avenue,NaN,New York,10016,NaN,MANHATTAN,40.741923,-73.977933,530 Second Ave,None,MANHATTAN,New York,NY,False,530 SECOND AVE||NEW YORK|NY|,"530 SECOND AVENUE, New York, NY, USA","530 Second Ave, MANHATTAN, NY",1009350005,D7,935.0,1.0,MN,04,7.0,51.0,7.0
1,6260224.0,3053070045,70 Dahill Road,NaN,Brooklyn,11218,NaN,BROOKLYN,40.645638,-73.981046,70 Dahill Rd,None,BROOKLYN,Brooklyn,NY,False,70 DAHILL RD||BROOKLYN|NY|,"70 DAHILL ROAD, Brooklyn, NY, USA","70 Dahill Rd, BROOKLYN, NY",3053070045,D1,5307.0,3.0,BK,03,6.0,122.0,6.0
2,6260314.0,1020910048,676 Riverside Drive,NaN,New York,10031,NaN,MANHATTAN,40.826851,-73.952170,676 Riverside Dr,None,MANHATTAN,New York,NY,False,676 RIVERSIDE DR||NEW YORK|NY|,"676 RIVERSIDE DRIVE, New York, NY, USA","676 Riverside Dr, MANHATTAN, NY",1020910048,D3,2091.0,1.0,MN,03,10.0,74.0,10.0
3,6272091.0,1021700292,320 Wadsworth Ave,NaN,NY,10040,NaN,MANHATTAN,40.854999,-73.931202,320 Wadsworth Ave,None,MANHATTAN,New York,NY,False,320 WADSWORTH AVE||NEW YORK|NY|,"320 WADSWORTH AVENUE, New York, NY, USA","320 Wadsworth Ave, MANHATTAN, NY",1021700292,D7,2170.0,1.0,MN,04,6.0,80.0,6.0
4,2810721.0,3060830041,9108 Colonial Road,NaN,Brooklyn,11209,NaN,BROOKLYN,40.621136,-74.037902,9108 Colonial Rd,None,BROOKLYN,Brooklyn,NY,False,9108 COLONIAL RD||BROOKLYN|NY|,"9108 COLONIAL ROAD, Brooklyn, NY, USA","9108 Colonial Rd, BROOKLYN, NY",3060830041,D1,6083.0,3.0,BK,03,6.0,69.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35913,NaN,2036530001,1560 Bruckner Blvd,NaN,Bronx,10473,BRONX,BRONX,40.823685,-73.875968,1560 Bruckner Blvd,None,BRONX,Bronx,NY,False,1560 BRUCKNER BLVD||BRONX|NY|,NaN,NaN,2036530001,U9,3653.0,2.0,BX,07,NaN,0.0,NaN
35914,NaN,2040420350,1610 Matthews Avenue,NaN,Bronx,10462,BRONX,BRONX,40.844451,-73.861078,1610 Matthews Ave,None,BRONX,Bronx,NY,False,1610 MATTHEWS AVE||BRONX|NY|,NaN,NaN,2040420350,U6,4042.0,2.0,BX,07,NaN,0.0,NaN
35915,NaN,1008460060,867 Broadway,NaN,New York,10003,NEW YORK,MANHATTAN,40.737567,-73.990207,867 Broadway,None,MANHATTAN,New York,NY,False,867 BROADWAY||NEW YORK|NY|,NaN,NaN,1008460060,K4,846.0,1.0,MN,04,5.0,0.0,5.0
35916,NaN,1010260041,210 W 55th St,NaN,New York,10019,NEW YORK,MANHATTAN,40.764491,-73.981651,210 W 55Th St,None,MANHATTAN,New York,NY,False,210 W 55TH ST||NEW YORK|NY|,NaN,NaN,1010260041,HB,1026.0,1.0,MN,05,13.0,4.0,13.0


In [18]:
# ll84_enriched.to_csv("building_go_bbl_pluto_enr.csv",index=False)
# ll84_enriched_old = pd.read_csv("building_go_bbl_pluto_enr.csv")

### Spatial joining with CT to pull ct into the buildings data. 

In [349]:
CT2010_URL = "https://data.cityofnewyork.us/resource/bmjq-373p.geojson?$limit=50000"
resp = requests.get(CT2010_URL, timeout=60)
resp.raise_for_status()

ct2010 = gpd.read_file(io.BytesIO(resp.content)).to_crs(2263)
ct2010.head()

,ntacode,shape_area,ntaname,shape_leng,boroname,puma,boroct2010,ct2010,borocode,cdeligibil,ctlabel,geometry
0,SI22,2497009.71359,West New Brighton-New Brighton-St. George,7729.01679383,Staten Island,3903,5000900,000900,5,E,9,"MULTIPOLYGON (((962269.126 173705.5, 962288.72..."
1,MN17,1860992.68163,Midtown-Midtown South,5687.80243891,Manhattan,3807,1010200,010200,1,I,102,"MULTIPOLYGON (((992216.539 216507.687, 992091...."
2,MN17,1864600.43538,Midtown-Midtown South,5693.03636707,Manhattan,3807,1010400,010400,1,I,104,"MULTIPOLYGON (((991325.882 217001.689, 991199...."
3,MN17,1890907.25105,Midtown-Midtown South,5699.86064037,Manhattan,3807,1011300,011300,1,I,113,"MULTIPOLYGON (((988650.277 214286.402, 988517...."
4,MN40,1918144.56374,Upper East Side-Carnegie Hill,5807.97295649,Manhattan,3805,1013000,013000,1,I,130,"MULTIPOLYGON (((994920.11 221386.27, 994791.85..."


### Adding CT 

In [350]:
# --- 1) Points from your geocoded table ---
# assumes working_residential_geo has columns: property_id, latitude, longitude
pts = gpd.GeoDataFrame(
    ll84_enriched[["property_id", "latitude", "longitude"]].copy(),
    geometry=gpd.points_from_xy(
        ll84_enriched["longitude"], ll84_enriched["latitude"]
    ),
    crs=4326  # your geocodes are WGS84
)

# project points to match the CT layer (your CT layer is already to_crs(2263))
pts_2263 = pts.to_crs(2263)

# --- 2) Keep only what you need from the CT layer ---
# 'ct2010' and/or 'boroct2010' are the usual tract IDs in that NYC layer
ct_keep = ct2010[["ct2010", "boroct2010", "boroname", "geometry"]].copy()

# --- 3) Spatial join: which tract polygon contains each point ---
joined = gpd.sjoin(
    pts_2263,
    ct_keep,
    how="left",
    predicate="within"   # points that fall inside a tract polygon
)

# If a property_id appears multiple times (duplicates), keep the first tract hit
joined = joined.sort_index().drop_duplicates(subset=["property_id"])

# --- 4) Bring tract columns back to your original dataframe ---
cols_to_add = ["ct2010", "boroct2010", "boroname"]
ll84_enriched_ct = ll84_enriched.merge(
    joined[["property_id"] + cols_to_add],
    on="property_id",
    how="left"
)

# working_with_tract now has ct2010 / boroct2010 for each row


In [351]:
ll84_enriched_ct

,property_id,bbl,address_1,address_2,city,postcode,county,borough,latitude,longitude,address_line1_clean,address_line2_clean,borough_clean,city_clean,state_clean,needs_geocoding,geocode_key,match_label,query,BBL,BldgClass,Block,BoroCode,Borough,LandUse,NumFloors,UnitsRes,stories,ct2010,boroct2010,boroname
0,2833318.0,1009350005,530 Second Avenue,NaN,New York,10016,NaN,MANHATTAN,40.741923,-73.977933,530 Second Ave,None,MANHATTAN,New York,NY,False,530 SECOND AVE||NEW YORK|NY|,"530 SECOND AVENUE, New York, NY, USA","530 Second Ave, MANHATTAN, NY",1009350005,D7,935.0,1.0,MN,04,7.0,51.0,7.0,007000,1007000,Manhattan
1,6260224.0,3053070045,70 Dahill Road,NaN,Brooklyn,11218,NaN,BROOKLYN,40.645638,-73.981046,70 Dahill Rd,None,BROOKLYN,Brooklyn,NY,False,70 DAHILL RD||BROOKLYN|NY|,"70 DAHILL ROAD, Brooklyn, NY, USA","70 Dahill Rd, BROOKLYN, NY",3053070045,D1,5307.0,3.0,BK,03,6.0,122.0,6.0,049800,3049800,Brooklyn
2,6260314.0,1020910048,676 Riverside Drive,NaN,New York,10031,NaN,MANHATTAN,40.826851,-73.952170,676 Riverside Dr,None,MANHATTAN,New York,NY,False,676 RIVERSIDE DR||NEW YORK|NY|,"676 RIVERSIDE DRIVE, New York, NY, USA","676 Riverside Dr, MANHATTAN, NY",1020910048,D3,2091.0,1.0,MN,03,10.0,74.0,10.0,022900,1022900,Manhattan
3,6272091.0,1021700292,320 Wadsworth Ave,NaN,NY,10040,NaN,MANHATTAN,40.854999,-73.931202,320 Wadsworth Ave,None,MANHATTAN,New York,NY,False,320 WADSWORTH AVE||NEW YORK|NY|,"320 WADSWORTH AVENUE, New York, NY, USA","320 Wadsworth Ave, MANHATTAN, NY",1021700292,D7,2170.0,1.0,MN,04,6.0,80.0,6.0,027900,1027900,Manhattan
4,2810721.0,3060830041,9108 Colonial Road,NaN,Brooklyn,11209,NaN,BROOKLYN,40.621136,-74.037902,9108 Colonial Rd,None,BROOKLYN,Brooklyn,NY,False,9108 COLONIAL RD||BROOKLYN|NY|,"9108 COLONIAL ROAD, Brooklyn, NY, USA","9108 Colonial Rd, BROOKLYN, NY",3060830041,D1,6083.0,3.0,BK,03,6.0,69.0,6.0,005201,3005201,Brooklyn
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35913,NaN,2036530001,1560 Bruckner Blvd,NaN,Bronx,10473,BRONX,BRONX,40.823685,-73.875968,1560 Bruckner Blvd,None,BRONX,Bronx,NY,False,1560 BRUCKNER BLVD||BRONX|NY|,NaN,NaN,2036530001,U9,3653.0,2.0,BX,07,NaN,0.0,NaN,000900,1000900,Manhattan
35914,NaN,2040420350,1610 Matthews Avenue,NaN,Bronx,10462,BRONX,BRONX,40.844451,-73.861078,1610 Matthews Ave,None,BRONX,Bronx,NY,False,1610 MATTHEWS AVE||BRONX|NY|,NaN,NaN,2040420350,U6,4042.0,2.0,BX,07,NaN,0.0,NaN,000900,1000900,Manhattan
35915,NaN,1008460060,867 Broadway,NaN,New York,10003,NEW YORK,MANHATTAN,40.737567,-73.990207,867 Broadway,None,MANHATTAN,New York,NY,False,867 BROADWAY||NEW YORK|NY|,NaN,NaN,1008460060,K4,846.0,1.0,MN,04,5.0,0.0,5.0,000900,1000900,Manhattan
35916,NaN,1010260041,210 W 55th St,NaN,New York,10019,NEW YORK,MANHATTAN,40.764491,-73.981651,210 W 55Th St,None,MANHATTAN,New York,NY,False,210 W 55TH ST||NEW YORK|NY|,NaN,NaN,1010260041,HB,1026.0,1.0,MN,05,13.0,4.0,13.0,000900,1000900,Manhattan


In [357]:
ll84_enriched_ct.to_csv("ll84_enriched_ct_SEMIFINAL.csv",index=False)

In [353]:
## Saving information to Csv 
# ll84_enriched_ct.to_csv("ll84_enriched_ct.csv",index=False)
# working_residential_tract = pd.read_csv("working_residential_tract.csv")

### Tree Canopy Joining via CT

In [355]:
# Path to your geodatabase (.gdb)
# https://github.com/CityOfNewYork/nyc-geo-metadata/blob/main/Metadata/Metadata_TreeCanopyChange.md
gdb_path = r"C:\Users\johnf\Downloads\Tree_Canopy_Change (1)\Tree_Canopy_Change\NYC_TreeCanopyChange_2010_2017.gdb"

# List all layers in the geodatabase
layers = fiona.listlayers(gdb_path)
print("Layers available:", layers)

# Read a specific layer into a GeoDataFrame
gdf = gpd.read_file(gdb_path, layer=layers[0])  # Replace [0] with your desired layer
print(gdf.head())

# Optional: save as shapefile or GeoJSON
# gdf.to_file("output.shp") 
# gdf.to_file("output.geojson", driver="GeoJSON")

canopy_change = gdf.copy()
canopy_change

Layers available: ['NYC_TreeCanopyChange_2010_2017']


C:\Python310\lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: organizePolygons() received a polygon with more than 100 parts. The processing may be really slow.  You can skip the processing by setting METHOD=SKIP, or only make it analyze counter-clock wise parts by setting METHOD=ONLY_CCW if you can assume that the outline of holes is counter-clock wise defined
  return ogr_read(


       Class  Shape_Length    Shape_Area  \
0  No Change    732.955182   5800.500001   
1  No Change   2079.456465  35005.875002   
2  No Change    228.557599   1984.500000   
3  No Change    125.726893    617.750000   
4  No Change     41.031601     90.750000   

                                            geometry  
0  MULTIPOLYGON (((1008879.93 272372.3, 1008878.4...  
1  MULTIPOLYGON (((1008749.43 271870.8, 1008748.9...  
2  MULTIPOLYGON (((1008704.43 272105.3, 1008701.9...  
3  MULTIPOLYGON (((1009015.43 272757.3, 1009013.4...  
4  MULTIPOLYGON (((1008794.93 270669.8, 1008781.4...  


,Class,Shape_Length,Shape_Area,geometry
0,No Change,732.955182,5800.500001,"MULTIPOLYGON (((1008879.93 272372.3, 1008878.4..."
1,No Change,2079.456465,35005.875002,"MULTIPOLYGON (((1008749.43 271870.8, 1008748.9..."
2,No Change,228.557599,1984.500000,"MULTIPOLYGON (((1008704.43 272105.3, 1008701.9..."
3,No Change,125.726893,617.750000,"MULTIPOLYGON (((1009015.43 272757.3, 1009013.4..."
4,No Change,41.031601,90.750000,"MULTIPOLYGON (((1008794.93 270669.8, 1008781.4..."
...,...,...,...,...
5692504,Loss,63.798210,286.735284,"MULTIPOLYGON (((1014610.46 188155.382, 1014610..."
5692505,Loss,63.429348,314.238467,"MULTIPOLYGON (((1014606.12 187969.835, 1014605..."
5692506,Loss,223.829388,2467.258600,"MULTIPOLYGON (((1014946.295 187646.6, 1014944...."
5692507,Loss,96.181802,538.668512,"MULTIPOLYGON (((1014656.958 187618.818, 101465..."


In [356]:
canopy_change

,Class,Shape_Length,Shape_Area,geometry
0,No Change,732.955182,5800.500001,"MULTIPOLYGON (((1008879.93 272372.3, 1008878.4..."
1,No Change,2079.456465,35005.875002,"MULTIPOLYGON (((1008749.43 271870.8, 1008748.9..."
2,No Change,228.557599,1984.500000,"MULTIPOLYGON (((1008704.43 272105.3, 1008701.9..."
3,No Change,125.726893,617.750000,"MULTIPOLYGON (((1009015.43 272757.3, 1009013.4..."
4,No Change,41.031601,90.750000,"MULTIPOLYGON (((1008794.93 270669.8, 1008781.4..."
...,...,...,...,...
5692504,Loss,63.798210,286.735284,"MULTIPOLYGON (((1014610.46 188155.382, 1014610..."
5692505,Loss,63.429348,314.238467,"MULTIPOLYGON (((1014606.12 187969.835, 1014605..."
5692506,Loss,223.829388,2467.258600,"MULTIPOLYGON (((1014946.295 187646.6, 1014944...."
5692507,Loss,96.181802,538.668512,"MULTIPOLYGON (((1014656.958 187618.818, 101465..."


### Spatial Join for LiDar Canopy and Buildings Lat Long for those in thwe Data from 2010 through 2017

In [359]:
# --- 1) Points from your buildings (WGS84) ---
shape_bldg = gpd.GeoDataFrame(
    ll84_enriched_ct[["property_id","bbl", "latitude", "longitude"]].copy(),
    geometry=gpd.points_from_xy(
        ll84_enriched_ct["longitude"], ll84_enriched_ct["latitude"]
    ),
    crs=4326
)

# --- 2) Put both layers in the SAME CRS (use the canopy CRS) ---
canopy = canopy_change[["Class", "geometry"]].copy()
if canopy.crs is None:
    # set if your canopy file didn’t come with a CRS; adjust if needed
    canopy = canopy.set_crs(4326)  # NYC StatePlane ft; change if different

shape_bldg = shape_bldg.to_crs(canopy.crs)

# (optional but recommended) fix invalid polygon rings that can break joins
canopy["geometry"] = canopy.buffer(0)

In [369]:
# 0) Make sure both layers share the SAME CRS
print("shape_bldg CRS:", shape_bldg.crs)
print("canopy CRS:", canopy.crs)

if canopy.crs != shape_bldg.crs:
    canopy = canopy.to_crs(shape_bldg.crs)

# 1) Compute polygon area (in projected units)
canopy["_poly_area"] = canopy.geometry.area

# 2) Spatial join: which canopy polygon contains each building point?
joined = gpd.sjoin(
    shape_bldg[["property_id", "bbl", "geometry"]],
    canopy[["Class", "_poly_area", "geometry"]],
    how="left",
    predicate="within"
)

# 3) If canopy polygons overlap and produce duplicates, keep the largest polygon
if joined.duplicated(["property_id", "bbl"]).any():
    joined = (
        joined
        .sort_values(["property_id", "bbl", "_poly_area"],
                     ascending=[True, True, False])  # or ascending=False for all
        .drop_duplicates(subset=["property_id", "bbl"])
    )

# 4) Bring canopy class back to your original table
ll84_enriched_ct_canopy = ll84_enriched_ct.merge(
    joined[["property_id", "bbl", "Class"]]
        .rename(columns={"Class": "canopy_change_class"}),
    on=["property_id", "bbl"],
    how="left"
)


shape_bldg CRS: EPSG:2263
canopy CRS: EPSG:2263


In [730]:
canopy

,Class,geometry,_poly_area
0,No Change,"POLYGON ((1008879.93 272372.3, 1008878.43 2723...",5800.500001
1,No Change,"POLYGON ((1008749.43 271870.8, 1008748.93 2718...",35005.875002
2,No Change,"POLYGON ((1008704.43 272105.3, 1008701.93 2721...",1984.500000
3,No Change,"POLYGON ((1009015.43 272757.3, 1009013.43 2727...",617.750000
4,No Change,"POLYGON ((1008794.93 270669.8, 1008781.43 2706...",90.750000
...,...,...,...
5692504,Loss,"POLYGON ((1014610.46 188155.382, 1014610.249 1...",286.604624
5692505,Loss,"POLYGON ((1014606.12 187969.835, 1014605.88 18...",314.046692
5692506,Loss,"POLYGON ((1014946.295 187646.6, 1014944.43 187...",2465.742701
5692507,Loss,"POLYGON ((1014656.958 187618.818, 1014657.247 ...",538.408990


In [370]:
ll84_enriched_ct_canopy["canopy_change_class"].unique()

array([nan, 'No Change', 'Gain', 'Loss'], dtype=object)

In [728]:
null_can = ll84_enriched_ct_canopy[ll84_enriched_ct_canopy["canopy_change_class"].isnull()]
print(null_can.shape)
non_null_can = ll84_enriched_ct_canopy[~ll84_enriched_ct_canopy["canopy_change_class"].isnull()]
print(non_null_can.shape)

(26403, 32)
(9515, 32)


### Need to do, fix nulls in the spatial join for the Canopy dats 

In [736]:
# ## Prepping to enr failed spatial joins
# ct_keep = ct2010[["ct2010", "boroct2010", "borocode", "boroname", "geometry"]].copy()
# canopy_change = canopy_change.to_crs(ct_keep.crs)

In [740]:
# canopy_with_ct = gpd.sjoin(
#     canopy_change,
#     ct_keep,
#     how="left",        # keep all canopy polygons
#     predicate="intersects"
# )

# # drop the join index column that sjoin adds
# canopy_with_ct = canopy_with_ct.drop(columns=["index_right"])
# canopy_with_ct

In [ ]:
# # 1) compute intersections and their area
# intersections = gpd.overlay(canopy_change, ct_keep, how="intersection")
# intersections["int_area"] = intersections.geometry.area

# # 2) For each canopy polygon, keep the tract with the largest overlap
# idx = (
#     intersections
#     .sort_values("int_area", ascending=False)
#     .groupby(intersections.index)  # canopy index
#     .head(1)
#     .index
# )

# canopy_with_ct_max = intersections.loc[idx].copy()


In [729]:
## Fixing Nulls with normal join
# ct2010n

,property_id,bbl,address_1,address_2,city,postcode,county,borough,latitude,longitude,address_line1_clean,address_line2_clean,borough_clean,city_clean,state_clean,needs_geocoding,geocode_key,match_label,query,BBL,BldgClass,Block,BoroCode,Borough,LandUse,NumFloors,UnitsRes,stories,ct2010,boroct2010,boroname,canopy_change_class
0,2833318.0,1009350005,530 Second Avenue,NaN,New York,10016,NaN,MANHATTAN,40.741923,-73.977933,530 Second Ave,None,MANHATTAN,New York,NY,False,530 SECOND AVE||NEW YORK|NY|,"530 SECOND AVENUE, New York, NY, USA","530 Second Ave, MANHATTAN, NY",1009350005,D7,935.0,1.0,MN,04,7.0,51.0,7.0,007000,1007000,Manhattan,NaN
1,6260224.0,3053070045,70 Dahill Road,NaN,Brooklyn,11218,NaN,BROOKLYN,40.645638,-73.981046,70 Dahill Rd,None,BROOKLYN,Brooklyn,NY,False,70 DAHILL RD||BROOKLYN|NY|,"70 DAHILL ROAD, Brooklyn, NY, USA","70 Dahill Rd, BROOKLYN, NY",3053070045,D1,5307.0,3.0,BK,03,6.0,122.0,6.0,049800,3049800,Brooklyn,NaN
2,6260314.0,1020910048,676 Riverside Drive,NaN,New York,10031,NaN,MANHATTAN,40.826851,-73.95217,676 Riverside Dr,None,MANHATTAN,New York,NY,False,676 RIVERSIDE DR||NEW YORK|NY|,"676 RIVERSIDE DRIVE, New York, NY, USA","676 Riverside Dr, MANHATTAN, NY",1020910048,D3,2091.0,1.0,MN,03,10.0,74.0,10.0,022900,1022900,Manhattan,NaN
3,6272091.0,1021700292,320 Wadsworth Ave,NaN,NY,10040,NaN,MANHATTAN,40.854999,-73.931202,320 Wadsworth Ave,None,MANHATTAN,New York,NY,False,320 WADSWORTH AVE||NEW YORK|NY|,"320 WADSWORTH AVENUE, New York, NY, USA","320 Wadsworth Ave, MANHATTAN, NY",1021700292,D7,2170.0,1.0,MN,04,6.0,80.0,6.0,027900,1027900,Manhattan,NaN
4,2810721.0,3060830041,9108 Colonial Road,NaN,Brooklyn,11209,NaN,BROOKLYN,40.621136,-74.037902,9108 Colonial Rd,None,BROOKLYN,Brooklyn,NY,False,9108 COLONIAL RD||BROOKLYN|NY|,"9108 COLONIAL ROAD, Brooklyn, NY, USA","9108 Colonial Rd, BROOKLYN, NY",3060830041,D1,6083.0,3.0,BK,03,6.0,69.0,6.0,005201,3005201,Brooklyn,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35913,NaN,2036530001,1560 Bruckner Blvd,NaN,Bronx,10473,BRONX,BRONX,40.823685,-73.875968,1560 Bruckner Blvd,None,BRONX,Bronx,NY,False,1560 BRUCKNER BLVD||BRONX|NY|,NaN,NaN,2036530001,U9,3653.0,2.0,BX,07,NaN,0.0,NaN,000900,1000900,Manhattan,NaN
35914,NaN,2040420350,1610 Matthews Avenue,NaN,Bronx,10462,BRONX,BRONX,40.844451,-73.861078,1610 Matthews Ave,None,BRONX,Bronx,NY,False,1610 MATTHEWS AVE||BRONX|NY|,NaN,NaN,2040420350,U6,4042.0,2.0,BX,07,NaN,0.0,NaN,000900,1000900,Manhattan,NaN
35915,NaN,1008460060,867 Broadway,NaN,New York,10003,NEW YORK,MANHATTAN,40.737567,-73.990207,867 Broadway,None,MANHATTAN,New York,NY,False,867 BROADWAY||NEW YORK|NY|,NaN,NaN,1008460060,K4,846.0,1.0,MN,04,5.0,0.0,5.0,000900,1000900,Manhattan,NaN
35916,NaN,1010260041,210 W 55th St,NaN,New York,10019,NEW YORK,MANHATTAN,40.764491,-73.981651,210 W 55Th St,None,MANHATTAN,New York,NY,False,210 W 55TH ST||NEW YORK|NY|,NaN,NaN,1010260041,HB,1026.0,1.0,MN,05,13.0,4.0,13.0,000900,1000900,Manhattan,NaN


In [371]:
# ll84_enriched_ct_canopy.to_csv("ll84_enriched_ct_canopy.csv",index=False)

In [373]:
### Before snapping seeing how many data poitns are null , not have a canopy value
ll84_enriched_ct_canopy.groupby(["canopy_change_class"]).agg({"property_id":"nunique","bbl":"nunique"})

,property_id,bbl
canopy_change_class,,
Gain,1270,769
Loss,244,178
No Change,1871,1199


In [401]:
## Limited ll84_enriched_ct_canopy data for joining with the energy metrics
# ll84_enriched_ct_canopy.columns
ll84_enriched_ct_canopy[ll84_enriched_ct_canopy["borough"]=="STATEN IS"]="STATEN ISLAND"

## Striping those that are iffy, and keeping the solid ones. If needed i will clean later. 
print(ll84_enriched_ct_canopy.shape)
ll84_enriched_ct_canopy_limited = ll84_enriched_ct_canopy[(
    (ll84_enriched_ct_canopy["borough"]==ll84_enriched_ct_canopy["borough_clean"])
    &(## manhattan
        ((ll84_enriched_ct_canopy["bbl"].astype(str).str[0] == "1") & (ll84_enriched_ct_canopy["borough"]=="MANHATTAN"))
    ##Bronx
        |((ll84_enriched_ct_canopy["bbl"].astype(str).str[0] == "2") & (ll84_enriched_ct_canopy["borough"]=="BRONX"))
    #Brooklyn
         |((ll84_enriched_ct_canopy["bbl"].astype(str).str[0] == "3") & (ll84_enriched_ct_canopy["borough"]=='BROOKLYN'))
    #Queens
        |((ll84_enriched_ct_canopy["bbl"].astype(str).str[0] == "4") & (ll84_enriched_ct_canopy["borough"]=="QUEENS"))
    #Staten Island
    |((ll84_enriched_ct_canopy["bbl"].astype(str).str[0] == "5") & (ll84_enriched_ct_canopy["borough"]=="STATEN ISLAND"))
    # Null BBL
    |(ll84_enriched_ct_canopy["bbl"].isnull())
    ))]
print(ll84_enriched_ct_canopy_limited.shape)

### Dropped 1518 rows / buildings

(35918, 32)
(34400, 32)


In [402]:
### Before snapping seeing how many data poitns are null , not have a canopy value
ll84_enriched_ct_canopy_limited.groupby(["canopy_change_class"]).agg({"property_id":"nunique","bbl":"nunique"})

,property_id,bbl
canopy_change_class,,
Gain,1250,757
Loss,234,173
No Change,1816,1174


In [403]:
ll84_enriched_ct_canopy_limited.to_csv("ll84_enriched_ct_canopy_limited.csv",index=False)

## Have Building Information joined with and the Canopy information, need to join back with the energy usage data (LL84)for year over year data

In [661]:
### Limiting Energy Data tp those that have "Whole Building" metered eneryg area, but extrapolating for earlier years. 
residential3_2010_2017_lim1 = residential3_2010_2017[(
    ((residential3_2010_2017["metered_areas_energy"]=="Whole Building")&(residential3_2010_2017["year_ending_year"]>=2012))
     |(((residential3_2010_2017["metered_areas_energy"]=="Whole Building")|(residential3_2010_2017["metered_areas_energy"].isnull()))
       &(residential3_2010_2017["year_ending_year"]<=2012))
    )]
### Ensuring no relevant alerts for Energy Related meters 
residential3_2010_2017_lim2 = residential3_2010_2017_lim1[(
    ((residential3_2010_2017_lim1['alert_energy_no_meters'].isnull())|(residential3_2010_2017_lim1['alert_energy_no_meters']=='Ok'))
    &((residential3_2010_2017_lim1['alert_energy_meter_has_less'].isnull())|(residential3_2010_2017_lim1['alert_energy_meter_has_less']=='Ok'))
    &((residential3_2010_2017_lim1['alert_energy_meter_has_gaps'].isnull())|(residential3_2010_2017_lim1['alert_energy_meter_has_gaps']=='Ok'))
    &((residential3_2010_2017_lim1['alert_energy_meter_has'].isnull())|(residential3_2010_2017_lim1['alert_energy_meter_has']=='Ok'))
    &((residential3_2010_2017_lim1['alert_energy_meter_has_single'].isnull())|(residential3_2010_2017_lim1['alert_energy_meter_has_single']=='Ok'))
    &((residential3_2010_2017_lim1['alert_meter_has_less_than'].isnull())|(residential3_2010_2017_lim1['alert_meter_has_less_than']=='Ok'))
    &((residential3_2010_2017_lim1['alert_no_meters_are_associated'].isnull())|(residential3_2010_2017_lim1['alert_no_meters_are_associated']=='Ok'))
    &((residential3_2010_2017_lim1['alert_meter_has_overlaps'].isnull())|(residential3_2010_2017_lim1['alert_meter_has_overlaps']=='Ok'))
)]
### Dropping filtered columns 
residential3_2010_2017_lim2 = residential3_2010_2017_lim2.drop(columns=["metered_areas_energy","alert_energy_no_meters","alert_energy_meter_has_less",
                                                                        "alert_energy_meter_has_gaps","alert_energy_meter_has","alert_energy_meter_has_single",
                                                                        "alert_meter_has_less_than","alert_no_meters_are_associated","alert_meter_has_overlaps"])

### limiting to where number_of_buildings is 1 to keep simple, some years onlu have null values. 
residential3_2010_2017_lim3 = residential3_2010_2017_lim2[(
    ((residential3_2010_2017_lim2["number_of_buildings"]==1)&(residential3_2010_2017_lim2['year_ending_year']>=2013))
    |(((residential3_2010_2017_lim2["number_of_buildings"]==1)|(residential3_2010_2017_lim2["number_of_buildings"].isnull()))
      &(residential3_2010_2017_lim2['year_ending_year']<2013))
)]
### Generalized Way to limit to mutifamily residential buildings
multifamily_residential_cols = [i for i in residential3_2010_2017_lim3.columns if 'multifamily_' in i]
residential3_2010_2017_lim4 = residential3_2010_2017_lim3.dropna(subset=multifamily_residential_cols, how="all")

### Further Facility Type / Property Use Limiting
residential3_2010_2017_lim4 = residential3_2010_2017_lim4[(
    ((residential3_2010_2017_lim4["year_ending_year"]<2012) & (residential3_2010_2017_lim4["facility_type"]=='Multifamily Housing'))
    |((residential3_2010_2017_lim4["year_ending_year"]>=2012)&(residential3_2010_2017_lim4["facility_type"].isnull()))
)]

### Only Non estimated data, but also many nulls in this field
residential3_2010_2017_lim4 = residential3_2010_2017_lim4[(
    ((residential3_2010_2017_lim4['estimated_data_flag'].astype(str).str.upper().str.strip()=='NO')
     |(residential3_2010_2017_lim4['estimated_data_flag'].isnull()))
    &((residential3_2010_2017_lim4["temporary_data_flag"].isnull())
      |(residential3_2010_2017_lim4["temporary_data_flag"].astype(str).str.upper().str.strip()=='NO'))
    &((residential3_2010_2017_lim4["estimated_data_flag_1"].isnull())
      |(residential3_2010_2017_lim4["estimated_data_flag_1"].astype(str).str.upper().str.strip()=='NO'))
    &((residential3_2010_2017_lim4["estimated_values_energy"].isnull())
      |(residential3_2010_2017_lim4["estimated_values_energy"].astype(str).str.upper().str.strip()=='NO'))
    &((residential3_2010_2017_lim4["alert_property_has_no_uses"].isnull())
      |(residential3_2010_2017_lim4["alert_property_has_no_uses"].astype(str).str.upper().str.strip()=='OK'))
    &((residential3_2010_2017_lim4["default_values"].isnull())
      |(residential3_2010_2017_lim4["default_values"].astype(str).str.upper().str.strip()=='NO'))
    &((residential3_2010_2017_lim4["temporary_values"].isnull())
      |(residential3_2010_2017_lim4["temporary_values"].astype(str).str.upper().str.strip()=='NO'))
    
)]

## Dropping more columns already filtered on, but not needed for this analysis, also other columns not needed 9Less Noise to sift through for consolidation)
water_cols = [i for i in residential3_2010_2017_lim4.columns if 'water' in i]
national_metric_cols = [i for i in residential3_2010_2017_lim4.columns if 'national' in i]
flag_cols = [i for i in residential3_2010_2017_lim4.columns if 'flag' in i]
drinking_water = [i for i in residential3_2010_2017_lim4.columns if 'potable' in i]
electricity = [i for i in residential3_2010_2017_lim4.columns if 'electricity' in i] # looking at total energy for buildings not elec. (varies percentage by building)
other_specific_types = [i for i in residential3_2010_2017_lim4.columns if ('fuel' in i or 'steam' in i or 'diesel' in i or 'natural' in i )]
financial = [i for i in residential3_2010_2017_lim4.columns if ('_cost' in i or 'cost_' in i or 'savings' in i )]
greentech = [i for i in residential3_2010_2017_lim4.columns if 'green' in i]
emissions = [i for i in residential3_2010_2017_lim4.columns if 'emission' in i]


residential3_2010_2017_lim4 = residential3_2010_2017_lim4.drop(columns=list(set(["number_of_buildings",'estimated_values_energy']
                                                                                +multifamily_residential_cols
                                                                                +water_cols
                                                                                +national_metric_cols
                                                                                +flag_cols
                                                                                + drinking_water
                                                                                + electricity
                                                                                +other_specific_types
                                                                                +financial
                                                                                +greentech
                                                                                +emissions
                                                                               )))
## Droping all that are just null values for all entries
residential3_2010_2017_lim4 = residential3_2010_2017_lim4.dropna(how='all',axis=1)


In [664]:
print(residential3_2010_2017_lim4.shape)
residential3_2010_2017_lim4.groupby(["year_ending_year"]).agg({"property_id":"nunique",'bbl':"nunique"})

(46993, 125)


,property_id,bbl
year_ending_year,,
2010,10812,6077
2011,12797,7830
2012,527,503
2013,3555,3433
2014,1722,1680
2015,2636,2556
2016,3627,3519
2017,4186,3982


In [699]:
### Limtiing the residential3_2010_2017_lim4 df to those instances that buildinga are present for ALL 7 years
property_id_2010 = residential3_2010_2017_lim4["property_id"][((residential3_2010_2017_lim4['year_ending_year']==2010)
                                                               &(~residential3_2010_2017_lim4["property_id"].isnull()))].unique()
bbl_2010 = residential3_2010_2017_lim4["bbl"][((residential3_2010_2017_lim4['year_ending_year']==2010)
                                                               &(~residential3_2010_2017_lim4["bbl"].isnull()))].unique()
consistant_presence = []
for year in residential3_2010_2017_lim4["year_ending_year"].unique():
    print(year)
    temp_lim_year = residential3_2010_2017_lim4[residential3_2010_2017_lim4["year_ending_year"]==year].dropna(how='all',axis=1)
    print(temp_lim_year.shape)
    if year ==2010:
        consistant_presence.append(temp_lim_year)
        continue
    else:
        temp_lim = temp_lim_year[((temp_lim_year["bbl"].isin(bbl_2010))|(temp_lim_year["property_id"].isin(property_id_2010)))]
        print(temp_lim.shape)
        print(f" Dropped {temp_lim_year.shape[0]-temp_lim.shape[0]} rows")
        consistant_presence.append(temp_lim)
all_building_energy_2010_2017 = pd.concat(consistant_presence)

2017
(5384, 63)
(3771, 63)
 Dropped 1613 rows
2016
(3636, 59)
(2599, 59)
 Dropped 1037 rows
2015
(2640, 55)
(1854, 55)
 Dropped 786 rows
2014
(1726, 63)
(1269, 63)
 Dropped 457 rows
2013
(3629, 53)
(2631, 53)
 Dropped 998 rows
2012
(527, 43)
(311, 43)
 Dropped 216 rows
2011
(15680, 46)
(10722, 46)
 Dropped 4958 rows
2010
(13771, 43)


In [705]:
## Finalizgin the Column names 
all_building_energy_2010_2017[all_building_energy_2010_2017['year_ending_year']==2010].dropna(how='all',axis=1).columns

Index(['property_id', 'address_1', 'city', 'weather_normalized_site_eui',
       'borough', 'latitude', 'longitude', 'community_board',
       'council_district', 'source_years', 'source_api_url', 'source_info_url',
       'census_tract', 'nta', 'postcode', 'address_2', 'county', 'bbl',
       'year_ending_year', 'release_date', 'other_weekly_operating_hours',
       'bin', 'facility_name', 'rating', 'total_floor_space_sq_ft',
       'facility_type', 'last_modified_date', 'eligibility_for_energy_star',
       'energy_star_application_status', 'current_rating_1_100',
       'target_rating_1_100', 'current_energy_period_ending',
       'current_site_energy_intensity', 'current_total_site_energy',
       'target_site_energy_intensity', 'current_source_energy',
       'current_total_source_energy', 'current_weather_normalized',
       'total_indoor_and_outdoor', 'current_total_on_site', 'building_id',
       'period_ending_date', 'state'],
      dtype='object')

In [700]:
rename_2010 = {
    # Building area → canonical GFA
    "total_floor_space_sq_ft": "gfa_building_ft2",

    # Energy: EUI & totals
    "current_site_energy_intensity": "site_eui_kbtu_ft",
    "current_total_site_energy": "site_energy_use_kbtu",
    "target_site_energy_intensity": "target_site_eui_kbtu_ft",
    "current_total_source_energy": "source_energy_use_kbtu",

    # Normalized site energy index (not kBtu, but useful as separate indicator)
    "current_weather_normalized": "weather_normalized_site_energy_index",

    # Water & on-site energy
    "total_indoor_and_outdoor": "total_water_use_kgal",
    "current_total_on_site": "on_site_energy_use_kbtu",
    # keep current_source_energy as-is or drop it; you already have source_energy_use_kbtu above
}
rename_2011 = {
    # Building area
    "total_floor_space_sq_ft": "gfa_building_ft2",

    # Last modified → canonical
    "last_modified_date": "last_modified_date_property",

    # Energy: EUI & totals
    "current_site_energy_intensity": "site_eui_kbtu_ft",
    "current_total_site_energy": "site_energy_use_kbtu",
    "target_site_energy_intensity": "target_site_eui_kbtu_ft",
    "current_total_source_energy": "source_energy_use_kbtu",

    # Normalized site energy index
    "current_weather_normalized": "weather_normalized_site_energy_index",

    # Optional: you can keep current_source_energy as-is or ignore it
}
rename_2012 = {
    # State field
    "state_province": "state",

    # Building area
    "property_floor_area_buildings": "gfa_building_ft2",

    # Last modified → canonical
    "date_property_last_modified": "last_modified_date_property",

    # Energy fields already use canonical names:
    #   site_eui_kbtu_ft
    #   weather_normalized_site_eui
    #   site_energy_use_kbtu
    #   source_eui_kbtu_ft
    #   weather_normalized_source
    #   source_energy_use_kbtu
    #   target_site_eui_kbtu_ft
}
rename_2013 = {
    # Building area
    "property_floor_area_building": "gfa_building_ft2",

    # Last modified
    "date_property_last_modified": "last_modified_date_property",

    # Everything else you listed (energy) is already in canonical form:
    #   site_eui_kbtu_ft
    #   weather_normalized_site_eui
    #   site_energy_use_kbtu
    #   weather_normalized_site_energy
    #   source_eui_kbtu_ft
    #   weather_normalized_source
    #   source_energy_use_kbtu
    # plus the various weather_normalized_site* convenience columns
}
rename_2014 = {
    # Building area
    "property_floor_area_building": "gfa_building_ft2",

    # Energy fields already canonical names as in 2013
}
rename_2015 = {
    # Building area: self-reported & EPA-calculated
    "property_gfa_self_reported": "gfa_building_ft2",
    "property_gfa_epa_calculated": "gfa_epa_total_ft2",       # bldg + parking
    "property_gfa_epa_calculated_1": "gfa_epa_buildings_ft2", # buildings only

    # Last modified
    "date_property_last_modified": "last_modified_date_property",

    # (Energy columns already have canonical names)
}
rename_2016 = {
    # Building area: self-reported & calculated
    "property_gfa_self_reported": "gfa_building_ft2",
    "property_gfa_calculated": "gfa_epa_total_ft2",       # equivalent to EPA-calculated total
    "property_gfa_calculated_1": "gfa_epa_buildings_ft2", # buildings only

    # Last modified
    "date_property_last_modified": "last_modified_date_property",

    # Energy:
    #   site_eui_kbtu_ft
    #   weather_normalized_site_eui
    #   site_energy_use_kbtu
    #   weather_normalized_site_energy
    #   source_eui_kbtu_ft
    #   weather_normalized_source
    #   source_energy_use_kbtu
    #   weather_normalized_source_1 (duplicate variant)
    #   source_energy_use_adjusted
    #   source_eui_adjusted_to_current
    # all already match canon → no rename needed
}
rename_2017 = {
    # Building area
    "property_gfa_self_reported": "gfa_building_ft2",
    "property_gfa_calculated": "gfa_epa_total_ft2",
    "property_gfa_calculated_1": "gfa_epa_buildings_ft2",

    # Last modified already comes as last_modified_date_property for 2017 in your list.
    # No rename required.

    # Energy fields already match 2016’s canonical naming.
}
rename_maps = {
    2010: rename_2010,
    2011: rename_2011,
    2012: rename_2012,
    2013: rename_2013,
    2014: rename_2014,
    2015: rename_2015,
    2016: rename_2016,
    2017: rename_2017,
}


In [707]:
# loop over years, rename columns, and collect
aligned_frames = []

for year in sorted(all_building_energy_2010_2017["year_ending_year"].unique()):
    print("Processing year:", year)
    temp_lim_year = all_building_energy_2010_2017[
        all_building_energy_2010_2017["year_ending_year"] == year
    ].copy()

    # apply the year-specific renames (if we have one)
    rename_dict = rename_maps.get(year, {})
    temp_lim_year = temp_lim_year.dropna(how='all',axis=1).rename(columns=rename_dict)
    if temp_lim_year.columns.duplicated().any():
        break
    print(temp_lim_year.columns)
    aligned_frames.append(temp_lim_year)

# 3) stitch them back together
all_building_energy_2010_2017_renamed = pd.concat(aligned_frames)

Processing year: 2010
Index(['property_id', 'address_1', 'city', 'weather_normalized_site_eui',
       'borough', 'latitude', 'longitude', 'community_board',
       'council_district', 'source_years', 'source_api_url', 'source_info_url',
       'census_tract', 'nta', 'postcode', 'address_2', 'county', 'bbl',
       'year_ending_year', 'release_date', 'other_weekly_operating_hours',
       'bin', 'facility_name', 'rating', 'gfa_building_ft2', 'facility_type',
       'last_modified_date', 'eligibility_for_energy_star',
       'energy_star_application_status', 'current_rating_1_100',
       'target_rating_1_100', 'current_energy_period_ending',
       'site_eui_kbtu_ft', 'site_energy_use_kbtu', 'target_site_eui_kbtu_ft',
       'current_source_energy', 'source_energy_use_kbtu',
       'weather_normalized_site_energy_index', 'total_water_use_kgal',
       'on_site_energy_use_kbtu', 'building_id', 'period_ending_date',
       'state'],
      dtype='object')
Processing year: 2011
Index(['pro

In [709]:
final_cols = [
    # Core / IDs / location
    "year_ending_year",
    "property_id",
    "bbl",
    "address_1",
    "address_2",
    "city",
    "borough",
    "postcode",
    "state",

    # Data / timing
    "release_date",
    "generation_date",
    "current_energy_period_ending",
    "energy_current_date",
    "period_ending_date",
    "last_modified_date_property",

    # Building
    "year_built",
    "gfa_building_ft2",
    "gfa_epa_total_ft2",
    "gfa_epa_buildings_ft2",
    "largest_property_use_type_1",
    "costar_property_id",

    # Energy – core
    "site_eui_kbtu_ft",
    "weather_normalized_site_eui",
    "site_energy_use_kbtu",
    "weather_normalized_site_energy",
    "source_eui_kbtu_ft",
    "weather_normalized_source",
    "source_energy_use_kbtu",
    "target_site_eui_kbtu_ft",

    # Energy – early-year extras
    "weather_normalized_site_energy_index",
    "current_source_energy",
    "on_site_energy_use_kbtu",
    "total_water_use_kgal",

    # Extra normalized variants
    "weather_normalized_source_1",
    "weather_normalized_site",
    "weather_normalized_site_1",
    "weather_normalized_site_2",
    "weather_normalized_site_3",

    # Adjusted source metrics
    "source_energy_use_adjusted",
    "source_eui_adjusted_to_current",
]

all_building_energy_2010_2017_renamed = all_building_energy_2010_2017_renamed[final_cols]
# all_building_energy_2010_2017_renamed.to_csv("all_building_energy_2010_2017_renamed.csv",index=False)

In [ ]:
## Finally Joining back in the buidlings information

In [ ]:
 	property_id 	bbl 	address_1 	address_2 	city

In [713]:
[i for i in all_building_energy_2010_2017_renamed.columns if i not in ['borough','postcode','state']]

['year_ending_year',
 'property_id',
 'bbl',
 'address_1',
 'address_2',
 'city',
 'release_date',
 'generation_date',
 'current_energy_period_ending',
 'energy_current_date',
 'period_ending_date',
 'last_modified_date_property',
 'year_built',
 'gfa_building_ft2',
 'gfa_epa_total_ft2',
 'gfa_epa_buildings_ft2',
 'largest_property_use_type_1',
 'costar_property_id',
 'site_eui_kbtu_ft',
 'weather_normalized_site_eui',
 'site_energy_use_kbtu',
 'weather_normalized_site_energy',
 'source_eui_kbtu_ft',
 'weather_normalized_source',
 'source_energy_use_kbtu',
 'target_site_eui_kbtu_ft',
 'weather_normalized_site_energy_index',
 'current_source_energy',
 'on_site_energy_use_kbtu',
 'total_water_use_kgal',
 'weather_normalized_source_1',
 'weather_normalized_site',
 'weather_normalized_site_1',
 'weather_normalized_site_2',
 'weather_normalized_site_3',
 'source_energy_use_adjusted',
 'source_eui_adjusted_to_current']

In [731]:
join_columns_energy = [i for i in all_building_energy_2010_2017_renamed.columns if i not in ['borough','postcode','state']]
join_columns_building = [i for i in ll84_enriched_ct_canopy_limited.columns if i not in ["borough","address_line1_clean","address_line2_clean","borough_clean",
                                                                  "city_clean","state_clean","needs_geocoding","match_label","query","BBL"]]
final_building_energy_data = all_building_energy_2010_2017_renamed[join_columns].merge(
    ### only with canopy values
    ll84_enriched_ct_canopy_limited[join_columns_building][~ll84_enriched_ct_canopy_limited['canopy_change_class'].isnull()],
        how="inner",
        on=["property_id","bbl","address_1","address_2","city"])





In [734]:
final_building_energy_data.groupby(['year_ending_year']).agg({"property_id":["nunique","count"],"bbl":"nunique"})

property_id           bbl
                     nunique count nunique
year_ending_year                          
2010                    2358  5058    1310
2011                    2007  4520    1096
2012                      51    95      51
2013                     567  1042     545
2014                     260   450     241
2015                     443   778     387
2016                     585   987     528
2017                     702  1489     615

In [739]:
final_building_energy_data.to_csv("final_building_energy_data.csv",index=False)

In [722]:
print(ll84_enriched_ct_canopy_limited[~ll84_enriched_ct_canopy_limited['canopy_change_class'].isnull()].shape)
print(ll84_enriched_ct_canopy_limited[ll84_enriched_ct_canopy_limited['canopy_change_class'].isnull()].shape)

(8483, 32)
(25917, 32)


## manual work scrp

In [654]:
### 
column_dict = {}
for year in residential3_2010_2017_lim4['year_ending_year'].unique():
    print(year)
    temp_df = residential3_2010_2017_lim4[residential3_2010_2017_lim4['year_ending_year']==year].dropna(how='all',axis=1)
    print(list(temp_df.columns))
    column_dict[year]=list(temp_df.columns)

2017
['property_id', 'property_name', 'year_ending', 'nyc_building_identification', 'address_1', 'city', 'largest_property_use_type_1', 'year_built', 'occupancy', 'energy_star_score', 'energy_star_certification', 'energy_star_certification_1', 'site_eui_kbtu_ft', 'weather_normalized_site_eui', 'site_energy_use_kbtu', 'weather_normalized_site_energy', 'source_eui_kbtu_ft', 'weather_normalized_source', 'source_energy_use_kbtu', 'weather_normalized_source_1', 'annual_maximum_demand_kw', 'annual_maximum_demand_mm', 'annual_maximum_demand_meter', 'percent_of_recs_retained', 'default_values', 'temporary_values', 'alert_property_has_no_uses', 'alert_data_center_issue_with', 'alert_gross_floor_area_is', 'property_gfa_self_reported', 'property_gfa_calculated', 'property_gfa_calculated_1', 'last_modified_date_property', 'last_modified_date_electric', 'last_modified_date_gas_meters', 'borough', 'latitude', 'longitude', 'community_board', 'council_district', 'last_modified_date_non', 'source_years

In [ ]:
### 2010
## Core for joining 
'year_ending_year',
'property_id',
'bbl',
'address_1',
'city',
"borough",
"postcode",
'address_2',
"state",
## Data Information
'release_date',
"last_modified_date",
"current_energy_period_ending",
"period_ending_date",
#Building
"total_floor_space_sq_ft",
## Energy Use
'current_energy_period_ending', 
'current_site_energy_intensity', 
'current_total_site_energy', 
'target_site_energy_intensity',
'current_source_energy', 
'current_total_source_energy', 
'current_weather_normalized', 
'total_indoor_and_outdoor', 
'current_total_on_site'
#---------------------------------------
## 2011
## Core for joining 
'year_ending_year',
'property_id',
'bbl',
'address_1',
'city',
"borough",
"postcode",
'address_2',
"state",
## Data Information
"release_date"
"current_energy_period_ending"

## building
"total_floor_space_sq_ft"
"year_built"
"last_modified_date"
# metering_configuration  ?

##energy
weather_normalized_site_eui
'current_energy_period_ending',
'current_site_energy_intensity',
'current_total_site_energy',
'target_site_energy_intensity',
'current_source_energy',
'current_total_source_energy',
'current_weather_normalized'
###-------------------------
##2012
'year_ending_year'
'property_id', 
'bbl'
 'address_1',
'city', 
 'borough',
 'address_2',
 'state_province'
## Data info
'generation_date'
'release_date', 
'energy_current_date'
#Building
'year_built',
'date_property_last_modified'
 'property_floor_area_buildings'

# Engergy
'site_eui_kbtu_ft',
'weather_normalized_site_eui',
'site_energy_use_kbtu',
'source_eui_kbtu_ft',
'weather_normalized_source',
'source_energy_use_kbtu',  
'target_site_eui_kbtu_ft',
### -------------------------
### 2013
## Core for joining
 'year_ending_year',
'property_id', 
 'bbl', 
'address_1', 
'city', 
 'borough',
'postcode',  
'address_2',
## data info 
'generation_date',
'release_date',

#building info
'year_built',  
 'date_property_last_modified'
'property_floor_area_building'
#energy
'site_eui_kbtu_ft',
'weather_normalized_site_eui',
'site_energy_use_kbtu',
'weather_normalized_site_energy',
'source_eui_kbtu_ft',
'weather_normalized_source',
'source_energy_use_kbtu',
'weather_normalized_source_1', 
'weather_normalized_site',
'weather_normalized_site_1',
  'costar_property_id', 
###-------------------------------
## 2014
## Core for joining 
'year_ending_year',
'property_id',
'bbl',
  'address_1', 
'city',
'borough',
'postcode'
'address_2'
## Data Information
'generation_date',
'release_date'

## Buidling Info
'year_built',
'property_floor_area_building',
## Energy Use
 'site_eui_kbtu_ft', 
'weather_normalized_site_eui', 
'site_energy_use_kbtu', 
'weather_normalized_site_energy',
'source_eui_kbtu_ft',
'weather_normalized_source'
'source_energy_use_kbtu'
 'weather_normalized_source_1'
'weather_normalized_site',
'weather_normalized_site_1',
'weather_normalized_site_2',
'weather_normalized_site_3',
## 2015
'property_id',
'address_1', 
'city',
 'borough',
 'postcode'
 'address_2', 
## Core for joining 

## Data Information
'generation_date'
'release_date',
## Buidling Info
'largest_property_use_type_1',
'year_built',
'date_property_last_modified',
"property_gfa_self_reported"
 'property_gfa_epa_calculated', 
'property_gfa_epa_calculated_1'
## Energy Use
'site_eui_kbtu_ft',
'weather_normalized_site_eui',
'site_energy_use_kbtu',
'weather_normalized_site_energy',
'source_eui_kbtu_ft',
'weather_normalized_source',
'source_energy_use_kbtu',
'weather_normalized_source_1', 
 'weather_normalized_site', 'weather_normalized_site_1', 'weather_normalized_site_2', 'weather_normalized_site_3',
#-----------------------------
## 2016
## Core for joining 
'property_id',
'bbl',
'address_1',
'city',
'borough'
 'postcode', 
'address_2',


## Data Information
'generation_date'
'release_date',
## Buidling Info
'largest_property_use_type_1',
'year_built',
 'property_gfa_self_reported', 
'property_gfa_calculated', 
'property_gfa_calculated_1',
'date_property_last_modified'
## Energy Use
'site_eui_kbtu_ft',
'weather_normalized_site_eui',
'site_energy_use_kbtu',
'weather_normalized_site_energy',
'source_eui_kbtu_ft',
'weather_normalized_source',
'source_energy_use_kbtu',
'weather_normalized_source_1',
 'weather_normalized_site', 
'weather_normalized_site_1', 
'weather_normalized_site_2', 
'weather_normalized_site_3',
'source_energy_use_adjusted'
'source_eui_adjusted_to_current'
##### --------------------------------------
## 2017
## Core for joining
 'year_ending_year'
'property_id', 
'bbl'
'address_1',
'city',  
'borough',
'postcode', 
'address_2',  \

## Data Information
'generation_date'

## Buidling Info
 'largest_property_use_type_1', 
'year_built',
'property_gfa_self_reported',
'property_gfa_calculated', 
'property_gfa_calculated_1
'last_modified_date_property',
## Energy Use
'site_eui_kbtu_ft', 
'weather_normalized_site_eui', 
'site_energy_use_kbtu',
'weather_normalized_site_energy',
'source_eui_kbtu_ft',
'weather_normalized_source',
'source_energy_use_kbtu',
'weather_normalized_source_1', 
 'weather_normalized_site', 
'weather_normalized_site_1',
'weather_normalized_site_2',
'weather_normalized_site_3', 
 'source_energy_use_adjusted',


## BELOW IS SCRAP -----------------------------------------

In [354]:
# Clean Borrough does not equal boro
fine = ll84_enriched_ct[((ll84_enriched_ct["borough"]==ll84_enriched_ct["borough_clean"])|
                         (ll84_enriched_ct["borough_clean"]=='STATEN ISLAND'))]
need_examination  = ll84_enriched_ct[((ll84_enriched_ct["borough"]!=ll84_enriched_ct["borough_clean"])
                        # S.I. is fine, manually checked excluding that brorough from ehre
                         &(ll84_enriched_ct["borough_clean"]!='STATEN ISLAND'))]
print(need_examination.shape)
need_examination.groupby(["borough","borough_clean","boro_code"]).agg({"property_id":"count"})

(842, 31)


KeyError: 'boro_code'

In [102]:
#Bronx Keeping
bronx_keeping = need_examination[((need_examination['borough']=="BRONX")
      &(need_examination["borough_clean"]=='MANHATTAN')
                 &(need_examination["match_label"].astype(str).str.contains("Bronx")))]
print(bronx_keeping.shape) ## Droping 4 instances of wrong entirs
#Brooklyn Keeping
brooklyn_keeping = need_examination[((need_examination['borough']=="BROOKLYN")
      &(need_examination["borough_clean"]=='MANHATTAN')
        &(need_examination["match_label"].astype(str).str.contains("Brooklyn")))]
print(brooklyn_keeping.shape)### Dropping 8 buildings
# QUEENS
queens_keeping1 = need_examination[((need_examination['borough']=="QUEENS")
      &(need_examination["borough_clean"]=='BROOKLYN')
      &(need_examination["match_label"].astype(str).str.contains("Far Rockaway")))]
queens_keeping2 = need_examination[((need_examination['borough']=="QUEENS")
      &(need_examination["borough_clean"]=='MANHATTAN')
                 &(need_examination["boro_code"]==4))]

## Dropping ~8 cols

(65, 34)
(18, 34)


In [129]:
examined_fine = pd.concat([bronx_keeping,brooklyn_keeping,queens_keeping1,queens_keeping2])
print(examined_fine.shape) ### overall dro[ping 20 rows
ll84_enriched_ct_clean1 = pd.concat([examined_fine,fine])

(112, 34)


In [130]:
pd.options.display.max_columns = 5400
## cleaning up the completely enriched building dataset
print(ll84_enriched_ct_clean1["confidence"].unique()) ### 80% and above so its all fine, and i can drop




### 
# ll84_enriched_ct

ll84_enriched_ct_clean1["raw_clean_address"] = ll84_enriched_ct_clean1["query"].combine_first(ll84_enriched_ct_clean1["geocode_key"])
ll84_enriched_ct_clean1["bbl"] = ll84_enriched_ct_clean1["BBL"].combine_first(ll84_enriched_ct_clean1["BBL_from_api"])
ll84_enriched_ct_clean1["address_line1_clean"] = ll84_enriched_ct_clean1["address_line1_clean"].combine_first(ll84_enriched_ct_clean1["address_1"])
ll84_enriched_ct_clean1["address_line2_clean"] = ll84_enriched_ct_clean1["address_line2_clean"].combine_first(ll84_enriched_ct_clean1["address_2"])
ll84_enriched_ct_clean1["city_clean"] = ll84_enriched_ct_clean1["city"].combine_first(ll84_enriched_ct_clean1["city_clean"])
ll84_enriched_ct_clean1["borough_clean"] = ll84_enriched_ct_clean1["borough"].combine_first(ll84_enriched_ct_clean1["borough_clean"])



to_drop = ["needs_geocoding","confidence","geocode_key","query","BBL_from_api",
           "BBL","BoroCode","Borough","boroname","block_num",
           "address_1", "address_2","city","borough","county"]
ll84_enriched_ct_clean2 = ll84_enriched_ct_clean1.drop(columns=to_drop)
ll84_enriched_ct_clean2 = ll84_enriched_ct_clean2.rename(columns = {"BIN_from_api":"BIN","match_label":"api_matched_address"})

[0.8 1. ]


In [132]:
# ll84_enriched_ct_clean2.to_csv("ll84_enriched_ct_clean2.csv",index=False)

In [134]:
ll84_enriched_ct_clean2.shape
### ~ 9,000 buildign universe to work with, pivoting to weaving in the canoopy tree data 

(9029, 21)

In [164]:
### last Trimming for BBL borocode not matching
##### WHILE Acriss seemed to confrim some of these were correct,
##### im being overtly cautious to have proper data. The borocode in the BBL shoudl match the location 
ll84_enriched_ct_clean3 = ll84_enriched_ct_clean2[(
    ## BRONX
    ((ll84_enriched_ct_clean2["borough_clean"]=="BRONX")
                         &(ll84_enriched_ct_clean2["boro_code"]==2))
    ## BROOKLYN
    |((ll84_enriched_ct_clean2["borough_clean"]=="BROOKLYN")
                         &(ll84_enriched_ct_clean2["boro_code"]==3))
    ## QUEENS
    |((ll84_enriched_ct_clean2["borough_clean"]=="QUEENS")
                         &(ll84_enriched_ct_clean2["boro_code"]==4))
        ## Statent Island
    |((ll84_enriched_ct_clean2["borough_clean"]=="STATEN IS")
                         &(ll84_enriched_ct_clean2["boro_code"]==5))
    ## Manhattan
    |((ll84_enriched_ct_clean2["borough_clean"]=="MANHATTAN")
                         &(ll84_enriched_ct_clean2["boro_code"]==1))
)]
ll84_enriched_ct_clean3["borough_clean"][ll84_enriched_ct_clean3["borough_clean"]=="STATEN IS"]="STATEN ISLAND"

C:\Users\johnf\AppData\Local\Temp\ipykernel_36636\3792486778.py:21: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  ll84_enriched_ct_clean3["borough_clean"][ll84_enriched_ct_clean3["borough_clean"]=="STATEN IS"]="STATEN ISLAND"
C:\Users\johnf\

In [160]:
ll84_enriched_ct_clean2[((ll84_enriched_ct_clean2["borough_clean"]=="STATENISLAND"))]
                         # &(ll84_enriched_ct_clean2["boro_code"]==5))]

,property_id,latitude,longitude,address_line1_clean,address_line2_clean,borough_clean,city_clean,state_clean,BIN,api_matched_address,boro_code,BldgClass,Block,LandUse,NumFloors,UnitsRes,stories,ct2010,boroct2010,raw_clean_address,bbl


In [161]:
ll84_enriched_ct_clean2
ll84_enriched_ct_clean2["borough_clean"].unique()

array(['BRONX', 'BROOKLYN', 'QUEENS', 'MANHATTAN', 'STATEN IS'],
      dtype=object)

In [166]:
ll84_enriched_ct_clean3.to_csv("ll84_enriched_ct_clean3.csv",index=False)
ll84_enriched_ct_clean3.shape

(8986, 21)

In [165]:
### 8800 is my final working set with confidence. 
ll84_enriched_ct_clean3.groupby(["borough_clean","boro_code"]).agg({"property_id":"count"})

,,property_id
borough_clean,boro_code,
BRONX,2,1709
BROOKLYN,3,1964
MANHATTAN,1,3479
QUEENS,4,1735
STATEN ISLAND,5,99


In [177]:
# --- 5) fill misses by snapping to nearest polygon within 50 ft ---
# This helps when a point lands just outside a sliver polygon.
miss_ids = ll84_enriched_ct_cln3_canopy.loc[ll84_enriched_ct_cln3_canopy["canopy_change_class"].isna(), "property_id"]
if len(miss_ids):
    nearest = gpd.sjoin_nearest(
        shape_bldg[shape_bldg["property_id"].isin(miss_ids)],
        canopy,
        how="left",
        max_distance=50  # units = CRS units; 50ft if CRS=2263
    )[["property_id", "Class"]].rename(columns={"Class": "canopy_change_class_nearest"})

    ll84_enriched_ct_cln3_canopy_50ft = ll84_enriched_ct_cln3_canopy.merge(nearest, on="property_id", how="left")
    ll84_enriched_ct_cln3_canopy_50ft["canopy_change_class"] = (
        ll84_enriched_ct_cln3_canopy_50ft["canopy_change_class"]
        .fillna(ll84_enriched_ct_cln3_canopy_50ft["canopy_change_class_nearest"])
    )
    ll84_enriched_ct_cln3_canopy_50ft.drop(columns=["canopy_change_class_nearest"], inplace=True)

In [178]:
ll84_enriched_ct_cln3_canopy_50ft.groupby(["canopy_change_class"]).agg({"property_id":"nunique"})

,property_id
canopy_change_class,
Gain,6024
Loss,971
No Change,1368


In [179]:
ll84_enriched_ct_cln3_canopy_50ft.shape

(9563, 22)

In [180]:
ll84_enriched_ct_cln3_canopy_50ft = ll84_enriched_ct_cln3_canopy_50ft.dropna(how='all', axis=1)
ll84_enriched_ct_cln3_canopy_50ft.to_csv("working_with_canopy_50ft.csv",index=False)

In [182]:
ll84_enriched_ct_cln3_canopy_50ft

,property_id,latitude,longitude,address_line1_clean,address_line2_clean,borough_clean,city_clean,state_clean,BIN,api_matched_address,boro_code,BldgClass,Block,LandUse,NumFloors,UnitsRes,stories,ct2010,boroct2010,raw_clean_address,bbl,canopy_change_class
0,2740249.0,40.840982,-73.854486,1705 Purdy St,NaN,BRONX,New York,NY,2096760.0,"1705 PURDY STREET, Bronx, NY, USA",2,R4,3944.0,03,13.0,3857.0,13.0,021001,2021001,"1705 Purdy St, MANHATTAN, NY",20394475010,Gain
1,2740271.0,40.840987,-73.854468,1718 Purdy St.,NaN,BRONX,New York,NY,2096761.0,"1718 PURDY STREET, Bronx, NY, USA",2,R4,3962.0,03,NaN,128.0,NaN,021001,2021001,"1718 Purdy St., MANHATTAN, NY",20396275010,Gain
2,2741284.0,40.839228,-73.858855,1501 Metropolitan Ave,NaN,BRONX,New York,NY,2096701.0,"1501 METROPOLITAN AVENUE, Bronx, NY, USA",2,R4,3944.0,03,13.0,3857.0,13.0,021001,2021001,"1501 Metropolitan Ave, MANHATTAN, NY",20394475010,Gain
3,2741286.0,40.840079,-73.857169,1591 Metropolitan Ave,NaN,BRONX,New York,NY,2096768.0,"1591 METROPOLITAN AVENUE, Bronx, NY, USA",2,R4,3944.0,03,13.0,3857.0,13.0,021001,2021001,"1591 Metropolitan Ave, MANHATTAN, NY",20394475010,Loss
4,2741298.0,40.838834,-73.860229,1 Metropolitan Oval,NaN,BRONX,New York,NY,2096667.0,"1 METROPOLITAN OVAL, Bronx, NY, USA",2,R4,3944.0,03,13.0,3857.0,13.0,021001,2021001,"1 Metropolitan Oval, MANHATTAN, NY",20394475010,Gain
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9558,3622045.0,40.787102,-73.787038,13-34 Estates Ln,NaN,QUEENS,Queens,NY,4130109.0,"13-34 ESTATES LANE, North Bayside, NY, USA",4,P5,5843.0,08,2.0,0.0,2.0,099705,4099705,"13-34 Estates Ln, QUEENS, NY",40584300200,Loss
9559,3630489.0,40.780374,-73.975764,44 West 77Th St,NaN,MANHATTAN,New York,NY,1028821.0,"44 WEST 77 STREET, New York, NY, USA",1,D4,1129.0,03,14.5,32.0,14.5,016100,1016100,"44 West 77Th St, MANHATTAN, NY",10112900550,Gain
9560,3633523.0,40.810108,-73.945088,128 West 128Th St,NaN,MANHATTAN,New York,NY,1089103.0,"128 WEST 128 STREET, New York, NY, USA",1,D1,1912.0,03,6.0,80.0,6.0,022400,1022400,"128 West 128Th St, MANHATTAN, NY",10191200430,Gain
9561,3633666.0,40.787834,-73.943766,315 E 102St St,NaN,MANHATTAN,New York,NY,1004271.0,"315 EAST HOUSTON STREET, New York, NY, USA",1,C3,350.0,02,4.0,4.0,4.0,016400,1016400,"315 E 102St St, MANHATTAN, NY",10035000610,Gain


In [207]:
### where is the main metric for energy use null
building_energy_info_canopy[((building_energy_info_canopy["weather_normalized_site_energy_use_kbtu"].isnull())
                             &(building_energy_info_canopy["weather_normalized_site_energy"].isnull())
                            )].shape

(6542, 38)

In [206]:
building_energy_info_canopy[((~building_energy_info_canopy["weather_normalized_site_energy_use_kbtu"].isnull())
                             |(~building_energy_info_canopy["weather_normalized_site_energy"].isnull())
                            )].shape

(26250, 38)

In [194]:

building_energy_info_canopy = lim_residential3_2012_2017.merge(ll84_enriched_ct_cln3_canopy_50ft, on = ["property_id"], how='inner')

In [197]:
## Working Data set for buioldings and energy. 
## Getting metrics
print(building_energy_info_canopy.shape)
## 
print("property _id")
print(len(building_energy_info_canopy["property_id"].unique()))
print("bbl")
print(len(building_energy_info_canopy["bbl"].unique()))

(32792, 38)
property _id
8635
bbl
8133


In [201]:
## Checking the property id / bbl relatinoship
test = building_energy_info_canopy.groupby(['bbl']).agg({"property_id":"nunique"}).reset_index()
test[test['property_id']>1]

,bbl,property_id
22,10001675090,2
53,10004200010,2
60,10005575010,3
70,10006975020,2
89,10012800260,3
...,...,...
8092,50059305900,4
8094,50059500150,9
8128,50398300010,2
8129,50398300450,3


In [203]:
## spot checking those 359 instances of PPls with more than one unsiue property 1i
building_energy_info_canopy[building_energy_info_canopy['bbl']=='10004200010']

,property_id,year_ending_year,weather_normalized_site_energy_use_kbtu,weather_normalized_site_energy,site_energy_use_kbtu,property_gfa_calculated_buildings_ft,property_gfa_epa_calculated,property_gfa_self_reported_ft,weather_normalized_site_eui_kbtu_ft,weather_normalized_site_eui,number_of_active_energy_meters_used_to_compute_metrics,alert_energy_no_meters_selected_for_metrics,alert_energy_meter_has_less_than_12_full_calendar_months_of_data,alert_energy_meter_has_overlaps,alert_energy_meter_has_gaps,alert_energy_meter_has_single_entry_more_than_65_days,alert_energy_no_meters_selected_for_metrics,latitude,longitude,address_line1_clean,address_line2_clean,borough_clean,city_clean,state_clean,BIN,api_matched_address,boro_code,BldgClass,Block,LandUse,NumFloors,UnitsRes,stories,ct2010,boroct2010,raw_clean_address,bbl,canopy_change_class
602,4039462.0,2017,NaN,25136390.2,24404561.5,NaN,NaN,NaN,NaN,89.9,NaN,NaN,NaN,NaN,Ok,NaN,NaN,40.706867,-74.00699,100 Maiden Ln,NaN,MANHATTAN,New York,NY,1001014.0,"100 MAIDEN LANE, New York, NY, USA",1,D6,42.0,04,24.0,340.0,24.0,000700,1000700,"100 Maiden Ln, MANHATTAN, NY",10004200010,Gain
7009,4039462.0,2016,NaN,23164416.9,22470279.9,NaN,NaN,NaN,NaN,82.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.706867,-74.00699,100 Maiden Ln,NaN,MANHATTAN,New York,NY,1001014.0,"100 MAIDEN LANE, New York, NY, USA",1,D6,42.0,04,24.0,340.0,24.0,000700,1000700,"100 Maiden Ln, MANHATTAN, NY",10004200010,Gain
12792,4039462.0,2015,NaN,24332716.1,24723102.1,NaN,279600.0,NaN,NaN,87.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.706867,-74.00699,100 Maiden Ln,NaN,MANHATTAN,New York,NY,1001014.0,"100 MAIDEN LANE, New York, NY, USA",1,D6,42.0,04,24.0,340.0,24.0,000700,1000700,"100 Maiden Ln, MANHATTAN, NY",10004200010,Gain
22563,4039462.0,2014,NaN,23872638.8,24959229.8,NaN,NaN,NaN,NaN,85.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.706867,-74.00699,100 Maiden Ln,NaN,MANHATTAN,New York,NY,1001014.0,"100 MAIDEN LANE, New York, NY, USA",1,D6,42.0,04,24.0,340.0,24.0,000700,1000700,"100 Maiden Ln, MANHATTAN, NY",10004200010,Gain
29207,3089482.0,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.706867,-74.00699,100 Maiden Ln,NaN,MANHATTAN,New York,NY,1001014.0,"100 MAIDEN LANE, New York, NY, USA",1,D6,42.0,04,24.0,340.0,24.0,000700,1000700,"100 Maiden Ln, MANHATTAN, NY",10004200010,Gain
31947,4039462.0,2013,NaN,21185029.9,21199896.6,NaN,NaN,NaN,NaN,75.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.706867,-74.00699,100 Maiden Ln,NaN,MANHATTAN,New York,NY,1001014.0,"100 MAIDEN LANE, New York, NY, USA",1,D6,42.0,04,24.0,340.0,24.0,000700,1000700,"100 Maiden Ln, MANHATTAN, NY",10004200010,Gain


In [ ]:
### Building Footprint data and Building Historic INFO and Height Ets
# BUILDING_HISTORIC
# https://data.cityofnewyork.us/City-Government/BUILDING_HISTORIC/ipkp-snf6/about_data

#Building Elevation and Subgrade (BES)
# https://data.cityofnewyork.us/City-Government/Building-Elevation-and-Subgrade-BES-/bsin-59hv/about_data